In [1]:
"""сокращаем импорт"""

from __future__ import division, print_function

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

"""дополнительный блок на решение прикладных задач"""

import datetime
import datetime as dt
from collections import Counter

"""без графиков нам больше не нужно"""

from sklearn.ensemble.forest import RandomForestRegressor

pd.set_option('display.max_columns', 64)

"""на предупреждение снизу внимание обращать не надо"""

'на предупреждение снизу внимание обращать не надо'

In [2]:
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.optimize import minimize

from sklearn.model_selection import TimeSeriesSplit

from statsmodels.iolib.table import SimpleTable
import tables

import matplotlib.pyplot as plt

from datetime import timedelta

In [2]:
# 30_04_2020 бек ап.

if df_n._ГарНомер.dtype == ('object'):
    df = df_n[df_n['_ГарНомер'].str.contains(name,na=False)] 
else:
    df = df_n[df_n['_ГарНомер']==name] 

In [3]:
"""препроцессинг

   03.03.2020 не трогай работает так как ты и задумывал"""

def eva(route_1,s_year,s_month,s_day,e_year,e_month,e_day,name):
    def build_df(df):
        
        # принимаем датафейм, убираем ошибки в логике формата представления дат вроде 0208 года нашей эры, вместо 2008 

        df['_Дата'] = pd.to_datetime(df['_Дата'], errors='coerce') 
        
        # пришлось добавить дома из за разницу версий
        
        df['_ДатаНАч'] = pd.to_datetime(df['_ДатаНАч'], errors='coerce') 
        df['_ДатаКон'] = pd.to_datetime(df['_ДатаКон'], errors='coerce') 

        # сбрасываем unnamed, если он затесался в датафрейм при чтении или при выгрузке данных, что иногда случается 

        df = df.drop(df.columns[df.columns.str.contains('unnamed', case = False)], axis = 1) 

        # принимаем датафейм, считаем пропуски, возвращаем mis_val_table_ren_columns, возвращает принт с описанием

        title = name

        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

        mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)

        start = df['_Дата'].min()
        end = df['_Дата'].max()
        print ('\n','\n',title, '\n', '\n', len(df),' ', 'Записей в отношении',' ',len(df._ТС.unique()),' ','моделей ТС', '\n', 
               "Начало записи данных ", start,'\n','Окончание записи данных ', end,'\n',
               "В наборе данных " + str(df.shape[1]) + " признаков.",'\n'     
                + str(mis_val_table_ren_columns.shape[0]) +
                  " признаков содержат пропуски.", sep='')   

        # маркируем пропуски в данных, которые описывают нарушения в учётной политике в отношении ремонтных работ

        df['_ВидОбслуживания'] = df['_ВидОбслуживания'].fillna('НАРУШЕНИЕ').astype('str') 
        df['_КатегорияРабот'] = df['_КатегорияРабот'].fillna('НАРУШЕНИЕ').astype('str')
        df['_ГруппаРабот'] = df['_ГруппаРабот'].fillna('НАРУШЕНИЕ').astype('str') 
        df['_Работа'] = df['_Работа'].fillna('НАРУШЕНИЕ').astype('str')
        df['_ФиоРММ'] = df['_ФиоРММ'].fillna('НАРУШЕНИЕ').astype('str')
        df['_ТС'] = df['_ТС'].fillna("НАРУШЕНИЕ").astype('str')  

        # приводим шрифты к верхнему регистру

        df['_ТС'] = df['_ТС'].str.upper()
        df['_ФиоРММ'] = df['_ФиоРММ'].str.upper() 
        df['_Работа'] = df['_Работа'].str.upper()
        df['_КатегорияРабот'] = df['_КатегорияРабот'].str.upper()
        df['_ВидОбслуживания'] = df['_ВидОбслуживания'].str.upper() 

        # формируем признак операци, в состава видов обслуживания, категории работ и работа  

        df['Операции'] = df['_ВидОбслуживания'] +" "+df['_КатегорияРабот'] +" "+ df['_Работа']
        
        # вот здесь надо попроавить ради системы предиктивной ананлитики - 31_07_2020

        # иногда Гаражные номера имеют представление ввиде числе с плавающей точкой, такие они Ямальсике 1С программисты 

        if df._ГарНомер.dtype == ('float64'):
            df['_ГарНомер'] = df['_ГарНомер'].astype('str')
            df['_ГарНомер'] = df['_ГарНомер'].map(lambda x: str(x)[:-2])
        else:
            df['_ГарНомер'] = df['_ГарНомер'].astype('str')

        # преобразуем типы данных в представление строк для последующей коннетации строк 

        df['_Номер'] = df['_Номер'].astype('str')
        df['_Дата'] = df['_Дата'].astype('str')

        # создаём дубликат для того что бы не трогать оргинальный признак ремонтного листа

        df['Машины'] = df['_ТС']

        # производим коннетациюю строк, формируем признак - уникальный гаражный номеро unic_gar

        df['Unic_gar'] = df['Машины'] + " " + df['_ГарНомер']

        # формируем признак год и уникальнй индентификатор ремонтного листа по годам, которого нет в 1с
        # result == номер ремонтного листа + номер года в котором этотм ремонтный лист сформирован, так то

        df['year'] = df['_Дата'].map(lambda x: str(x)[:-6])
        df['result'] = df["_Номер"] + df["year"]

        # создаём представление дат с формированием колонки delta - продолжительность ремонтного листа по листу
        
        """ пришлось выключить дома из-за разницы версий
        df['_ДатаНАч'] = df['_ДатаНАч'].astype('datetime64')
        df['_ДатаКон'] = df['_ДатаКон'].astype('datetime64')
        """
        df['delta'] = df['_ДатаКон'] - df['_ДатаНАч']

        # делаем первый бэкап

        df_backup = df.copy()   

        # заменяем пробелы на нижнее подчёркивание

        df =  df.replace(r'\s+','_',regex=True)

        # заменяем злоебучие скобки

        df = df.replace(to_replace='\(', value="", regex=True)
        df = df.replace(to_replace='\)', value="", regex=True)

        # добавляем функцию селекции, которая перебирает данные в основной витрине и возвращает датафреймы по годам

        def pony_select(df,s_year,s_month,s_day,e_year,e_month,e_day):
            """сбрасываем колонку year в годовых датафреймах, она там уже не нужна
            выводим в принт длинну выборки по годам - может пригодиться """
            df_2019 = df[df['year'].str.contains('2019',na=False)]
            df_2019 = df_2019.drop(['year'],axis=1)
            df_2018 = df[df['year'].str.contains('2018',na=False)]
            df_2018 = df_2018.drop(['year'],axis=1)
            df_2017 = df[df['year'].str.contains('2017',na=False)]
            df_2017 = df_2017.drop(['year'],axis=1)
            df_2016 = df[df['year'].str.contains('2016',na=False)]
            df_2016 = df_2016.drop(['year'],axis=1)
            print ('Количество записей по годам: ', '19:',len(df_2019) ,',','18:',
            len(df_2018),',','17:', len(df_2017),',','16:', len(df_2016))
            
            """но так криво с селектом мы работать не будем"""   
        
            df = df.drop(['year'],axis=1)
            df['_Дата'] = pd.to_datetime(df['_Дата'], errors='coerce') 
            start = datetime.datetime( s_year,s_month,s_day)
            end = datetime.datetime(e_year,e_month,e_day)
            mask = (df['_Дата'] > start) & (df['_Дата'] <= end)
            result = df.loc[mask]   
    
            return result,df_2019,df_2018,df_2017,df_2016  
    
        result, df_2019,df_2018,df_2017,df_2016 = pony_select(df,s_year,s_month,s_day,e_year,e_month,e_day)   

        # пока неплохо получается! добавляем фильтрацию, местами себя оправдывает.

        def hard(df):
            filtered_df= df[(df['_ДатаНАч'] <= df['_ДатаКон']) & 
                         (df['_ДатаНАч'] >= df['_ДатаКон'].quantile(0.025)) &
                         (df['_ДатаНАч'] >= df['_ДатаКон'].quantile(0.075)) &
                         (df['_ДатаНАч'] <= df['_ДатаКон'].quantile(0.925)) &
                         (df['_ДатаНАч'] <= df['_ДатаКон'].quantile(0.975)) &
                         (df['_ДатаКон'] >= df['_ДатаНАч'].quantile(0.025)) &
                         (df['_ДатаКон'] >= df['_ДатаНАч'].quantile(0.075)) &
                         (df['_ДатаКон'] <= df['_ДатаНАч'].quantile(0.925)) &
                         (df['_ДатаКон'] <= df['_ДатаНАч'].quantile(0.975))]
            return filtered_df

        def soft(df):
            filtered_df = df[(df['_ДатаНАч'] <= df['_ДатаКон']) & 
                         (df['_ДатаНАч'] >= df['_ДатаКон'].quantile(0.025)) &
                         (df['_ДатаНАч'] <= df['_ДатаКон'].quantile(0.975)) &
                         (df['_ДатаКон'] >= df['_ДатаНАч'].quantile(0.025)) &
                         (df['_ДатаКон'] <= df['_ДатаНАч'].quantile(0.975))]
            return filtered_df

        # исполняем фильтры    

        soft_19 = soft(df_2019)
        soft_18 = soft(df_2018)
        soft_17 = soft(df_2017)
        soft_16 = soft(df_2016)

        hard_19 = hard(df_2019)
        hard_18 = hard(df_2019)
        hard_17 = hard(df_2019)
        hard_16 = hard(df_2019)

        print("я лапочка, эта штука работает",'\n','\n') 

        return result, soft_19,soft_18,soft_17,soft_16, hard_19,hard_18,hard_17,hard_16, mis_val_table_ren_columns, df_backup

    def soft_stat(df):
        filtered_df = df[(df['_ДатаНАч'] <= df['_ДатаКон']) & 
                     (df['_ДатаНАч'] >= df['_ДатаКон'].quantile(0.025)) &
                     (df['_ДатаНАч'] <= df['_ДатаКон'].quantile(0.975)) &
                     (df['_ДатаКон'] >= df['_ДатаНАч'].quantile(0.025)) &
                     (df['_ДатаКон'] <= df['_ДатаНАч'].quantile(0.975))]
        array = filtered_df['delta'].values
        array = array.astype('timedelta64[D]')
        pony = Counter(array)
        anime = pony.most_common(10)
        soft_info = filtered_df['delta'].describe()
        print("Размер выборки",'\n',filtered_df.shape[0] / df.shape[0],'\n','\n',soft_info,
              '\n','\n', "Хит-парад наиболее регулярных периодов ремонта по убыванию", '\n','\n',
              "(1,'D'), 11) означает что период ремонта 1 день повторился 11 раз",'\n',anime,sep="")
        return anime, soft_info

    def call_all():    

        """откликаеться на nst, mis_val_table_ren_columns, pony, soft_info = call_all()
            name_1 - ксоплей sam"""

        df_1 = pd.read_csv(route_1, sep=';', encoding="utf-8")
        result,soft_19,soft_18,soft_17,soft_16, hard_19,hard_18,hard_17,hard_16, mis_val_table_ren_columns, df_backup = build_df(df_1)
        pony, soft_info = soft_stat(result) # пони и сфот инфо беруться по интервалу который мы задаём в фукнции путей

        return result, mis_val_table_ren_columns,pony, soft_info

    result, mis_val_table_ren_columns, pony, soft_info = call_all()  
    
    return (result, mis_val_table_ren_columns, pony, soft_info)

In [4]:
def eva_00():
    
    """      
    фукнция путей, не трогай работает так как ты и задумывал 03.03.2020
    
    даты записываються в формате YYYY MM DD, сначала дата начала потом дата конца
    
    откликаеться на result_utt,result_nst,result_trans,result_nvds,result_cher,result_mtc,result_mtk,result_sam = eva_00()
    
    """
    
    result_utt = eva("C:/Users/Battlestation/ural_2020/2020_UTT.csv",
                                 2019,1,1, 
                                 2020,1,1,
                                 'utt') 
    
    result_sam= eva("C:/Users/Battlestation/ural_2020/2020_SAM.csv",
                                 2019,1,1,
                                 2020,1,1,
                                 'sam') 
    
    result_mtk= eva("C:/Users/Battlestation/ural_2020/2020_MTK.csv",                                 
                                 2019,1,1,
                                 2020,1,1,
                                 'mtk') 
    
    result_mtc = eva("C:/Users/Battlestation/ural_2020/2020_MTS.csv",
                                 2019,5,5,
                                 2020,1,1,
                                 'mtc') # границы диапазона смотреть на принтах
    
    result_cher= eva("C:/Users/Battlestation/ural_2020/2020_CHER.csv",
                                    2019,1,1, 
                                    2020,1,1,
                                    'cher') # etc
    
    result_nvds = eva("C:/Users/Battlestation/ural_2020/2020_NVDS.csv",
                                    2019,1,1,
                                    2020,1,1,
                                    'nvds') # etc
    
    # по транссервису пока новых машин нет и диапазон дат нельзя задавать мимо кассы
    
    result_trans = eva("C:/Users/Battlestation/ural_2020/2020_TRANS.csv",
                                       2019,1,1,
                                       2020,1,1,
                                       'trans') 
        
    result_nst = eva( "C:/Users/Battlestation/ural_2020/2020_NST.csv",
                                 2019,1,1, 
                                 2019,12,31,
                                 'nst') 
    # вывод в файлы
    
    # мой домашний путь 18.03.2020 C:/Users/Battlestation/ural/UTT_OS.xls
    
    # мой путь в офисе C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_test/
    
    result_utt[0].to_csv('C:/Users/Battlestation/ural/result_utt.csv', 
                  encoding='utf-8', index=False)
    result_nst[0].to_csv('C:/Users/Battlestation/ural/result_nst.csv', 
                  encoding='utf-8', index=False)
    result_trans[0].to_csv('C:/Users/Battlestation/ural/result_trans.csv', 
                  encoding='utf-8', index=False)
    result_nvds[0].to_csv('C:/Users/Battlestation/ural/result_nvds.csv', 
                  encoding='utf-8', index=False)
    result_cher[0].to_csv('C:/Users/Battlestation/ural/result_cher.csv', 
                  encoding='utf-8', index=False)
    result_mtc[0].to_csv('C:/Users/Battlestation/ural/result_mtc.csv', 
                  encoding='utf-8', index=False)
    result_mtk[0].to_csv('C:/Users/Battlestation/ural/result_mtk.csv', 
                  encoding='utf-8', index=False)
    result_sam[0].to_csv('C:/Users/Battlestation/ural/result_sam.csv', 
                  encoding='utf-8', index=False)
    
    return result_utt,result_nst,result_trans,result_nvds,result_cher,result_mtc,result_mtk,result_sam

In [5]:
def eva_01():
    """работает, 
       допиливаем
       [0] - датафрейм
       [1] - сводка потерь
       [2] - хит парад ремонта
       [3] - простоя статистика
       
       не трогай 03.03.2020, откликаеться на 
       
       result_utt,result_nst,result_trans,result_nvds,result_cher,result_mtc,result_mtk,result_sam,result_loss = eva_01()
    
       result_loss - сводка потерь в учётной политике 
    """
    result_utt,result_nst,result_trans,result_nvds,result_cher,result_mtc,result_mtk,result_sam = eva_00()
    
    result_utt[1].columns = result_utt[1].columns+" УТТ"
    result_nst[1].columns = result_nst[1].columns+" НСТ"
    result_trans[1].columns = result_trans[1].columns+" ТРА"
    result_nvds[1].columns = result_nvds[1].columns+" НВДС"    
    result_cher[1].columns = result_cher[1].columns+" ЧЕР"
    result_mtc[1].columns = result_mtc[1].columns+" МТС"
    result_mtk[1].columns = result_mtk[1].columns +" МТК"
    result_sam[1].columns = result_sam[1].columns + " САМ"
           
    dfs = [result_utt[1],result_nst[1],result_trans[1],result_nvds[1],
           result_cher[1],result_mtc[1],result_mtk[1],result_sam[1]]
    
    result_loss = pd.concat(dfs,axis=1)
    
    result_loss.to_csv('C:/Users/Battlestation/ural/result_1c.csv', encoding='utf-8', index=True) 
    
    result_loss.to_html('C:/Users/Battlestation/ural/сводка_потерь_учёта_тест.html')    
    
    return result_utt, result_nst, result_trans, result_nvds, result_cher, result_mtc, result_mtk, result_sam

In [6]:
def eva_02():
    """функция хранимого генератора, который обеспечивает интерацию объектов для алгоритма счисления
       мы считаем каждую машину отдельно согласно ТЗ от производственного блока
       откликаеться на 
       
       df_n,warp,rout, = eva_02() 
       
       [0] - утт
       [1] - нст
       [2] - транссервис
       [3] - нвдс
       
       [4] - черногор
       [5] - мтс
       [6] - мтк
       [7] - сам 
       
       """
    
    result_utt,result_nst,result_trans,result_nvds,result_cher,result_mtc,result_mtk,result_sam = eva_01()
    
    # мой путь на работе C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/2020_OS/NST_OS.xls
    
    rout_utt = "C:/Users/Battlestation/ural_2020/UTT_OS.xls"                                 
    
    rout_sam =  "C:/Users/Battlestation/ural_2020/SAM_OS.xls"
    
    rout_mtk= "C:/Users/Battlestation/ural_2020/MTK_OS.xls"                                 
    
    rout_mts =  "C:/Users/Battlestation/ural_2020/mtc_OS.xls"
    
    rout_cher = "C:/Users/Battlestation/ural_2020/CHA_OS.xls"
    
    rout_nvds =  "C:/Users/Battlestation/ural_2020/nvds_OS.xls"
    
    rout_trans = "C:/Users/Battlestation/ural_2020/trans_os.xls"
        
    rout_nst = "C:/Users/Battlestation/ural_2020/NST_OS.xls"    
    

    def warp_field_generator(df):
        for pony in df.Unic_gar.unique():
            yield pony
            
    warp_utt = warp_field_generator(result_utt[0])
    warp_nst = warp_field_generator(result_nst[0])
    warp_trans = warp_field_generator(result_trans[0])
    warp_nvds = warp_field_generator(result_nvds[0])
    
    warp_cher = warp_field_generator(result_cher[0])
    warp_mtc = warp_field_generator(result_mtc[0])
    warp_mtk = warp_field_generator(result_mtk[0])
    warp_sam = warp_field_generator(result_sam[0])
    
    """нужно помнить что вывод фукнции снова сжимает размер вывода и обращение к объектам идёт по [x][n]
       
       df[1][1] - сводка потерь мтк
       df[0][1] - сводка потерь 
       """

    return (result_utt,result_nst,result_trans,result_nvds,
            result_cher,result_mtc,result_mtk,result_sam), (warp_utt,warp_nst,warp_trans,warp_nvds,
            warp_cher,warp_mtc,warp_mtk,warp_sam), (rout_utt,rout_nst,rout_trans,rout_nvds,
            rout_cher,rout_mts,rout_mtk,rout_sam)

In [7]:
def coub():
    """coub отпринтит что засунули
    
       result_loss выведен из вывода поскольку сохраняеться ранее в файле 
    
    """
    
    df_n,warp,rout = eva_02() # исправлено добавил result_loss в поток
    
    # 24.03.2020 храним промежуточные таблицы в хранилище,работает.
    
    df_n[0][0].to_csv('C:/Users/Battlestation/ural/df_n_utt.csv', 
                  encoding='utf-8', index=False)
    df_n[1][0].to_csv('C:/Users/Battlestation/ural/df_n_nst.csv', 
                  encoding='utf-8', index=False)
    df_n[2][0].to_csv('C:/Users/Battlestation/ural/df_n_trans.csv', 
                  encoding='utf-8', index=False)
    df_n[3][0].to_csv('C:/Users/Battlestation/ural/df_n_nvds.csv', 
                  encoding='utf-8', index=False)
    df_n[4][0].to_csv('C:/Users/Battlestation/ural/df_n_cher.csv', 
                  encoding='utf-8', index=False)
    df_n[5][0].to_csv('C:/Users/Battlestation/ural/df_n_mtc.csv', 
                  encoding='utf-8', index=False)
    df_n[6][0].to_csv('C:/Users/Battlestation/ural/df_n_mtk.csv', 
                  encoding='utf-8', index=False)
    df_n[7][0].to_csv('C:/Users/Battlestation/ural/df_n_sam.csv', 
                  encoding='utf-8', index=False)
        
    return df_n,warp,rout

In [8]:
def jail():
    
    """мы вас спасём от вируса (нет нам опять похуй)"""
    
    df_n,warp,rout = coub()

    def horus(df_n):
        result = df_n['_ГарНомер'].value_counts().reset_index().sort_values(by='index')
        result.rename(columns={'index': '_ГарНомер', '_ГарНомер': 'ремонты'}, inplace=True)
        result = pd.merge(result, df_n, how ='inner', on ='_ГарНомер')
        
        try:
            result['_ГарНомер'] =result['_ГарНомер'].astype('int64') # вот эта штука вносит хаос в int\str и ебёт мозг
        except ValueError:
            result['_ГарНомер'] =result['_ГарНомер'].astype('str')
        
        
        columns = ['_Дата','_ВидОбслуживания','_ДатаНАч',
                     '_ДатаКон','_ГруппаРабот','_КатегорияРабот',
                     '_Работа','_ФиоРММ','Операции','Машины', 'result','_Номер','Unic_gar']
        result.drop(columns, inplace=True, axis=1)
    
        return result

    def solid(df_n):

        def warp_field_generator(df_n):
            for pony in df_n._ГарНомер.unique():
                yield pony
        
        warp = warp_field_generator(df_n)
    
        return warp

    def anime(df_n,warp):
    # засовывать остальное нужно сюда

        for pony in warp:
            name = pony
            heresy = df_n[df_n['_ГарНомер']==name] 
            heresy['простой'] = heresy['delta'].sum()
        
        
            return heresy.head(1)

    def snake(df_n):

        df_n = horus(df_n)
        warp = solid(df_n)
        
        one = pd.DataFrame()
    
        def call(one,df_n,warp):
            try:
                columns =['delta']
                tablet = anime(df_n,warp)             # запуска фукнции по кругу
                tablet.drop(columns, inplace=True, axis=1)
                vision = pd.concat([one,tablet])                # коннетация
                one = vision.copy()                           # копирование  (кеширование не работает без копирования)
                return tablet,vision,one
            except ValueError:
                next        
            except TypeError:
                print('в базе данных кончились машины - алгоритм отработал все объекты счисления') 
            
        def car_number(df_n):
            count = df_n['_ГарНомер'].nunique(dropna=True)  # подсчёт уникальных номеров и
            return count

        op_time = car_number(df_n) # формирование длинны кеша
        
        for _ in range(op_time):
            table,vision,one = call(one,df_n,warp)    # служебная
        
        def lazy_me(df_n):                    # пост обработка
            lazy = df_n.copy()
            lazy = lazy.reset_index()
            del lazy['index']
            return lazy

        vision = lazy_me(vision)  
    
        return vision
    
    # 03_06_2020 боремся за то что бы utt работало
    
    utt = snake(df_n[0][0])
    
    nst = snake(df_n[1][0])
    trans = snake(df_n[2][0])
    nvds = snake(df_n[3][0])
    cher = snake(df_n[4][0])
    mtc = snake(df_n[5][0])
    mtk = snake(df_n[6][0])
    sam = snake(df_n[7][0])
        
    return (utt,nst,trans,nvds,cher,mtc,mtk,sam)


In [9]:
def lonely():
    
    my_life = jail()

    def ranking(df_n):
        
        # проблема 4455 и 4468 ВНИМАНИЕ
        
        # при дальнейшей работе в таблице классов были обнаружены записи, которые имеют None в поле активности 
        # а также в поле ремонта. Доля таких записей не определена, но она не очень велика - порядка 10%
        # впервые такие записи были найдены в NST, это может повлиять на работу всего алгоритма
        # но мы пока вынуждены принебречь этим поскольку времени на тесты нет.
        # устранимая погрешность работы классификатора - min 10%
        
    
        """
    
        границы.
    
        надёжность:    X(unique,25%, 25%,25%,25%)
    
        активность:    Y(unique,25%, 25%,25%,25%)
    
        класс объекта: Z(Xn1~Yn1, X|Y(n5<std), X|Y(n>std))
    
        """
        # этап 1 формирование понятия надёжности.
        
        result = df_n.copy()
        
        test = df_n.copy()

        # надёжность

        stat = result.ремонты.describe()

        # сломались 1 раз
        
        mask = result.ремонты <=stat[3]                                          
        single = result[mask]
        
        # классификатор 1 раз
        
        single['надёжность'] = np.where( (single.ремонты <=stat[3]), 0,1) 
        
        # от 0 до 25% случаев
        
        mask = (result.ремонты > stat[3]) & (result.ремонты <stat[4])                                          
        line_0_25 = result[mask]
        
        # классификатор 0-25
        
        line_0_25['надёжность'] = np.where( (line_0_25.ремонты >=stat[3]) & (line_0_25.ремонты <stat[4]), 1,2)     
        
        # от 25% до 50 % случаев
        
        mask = (result.ремонты >= stat[4]) & (result.ремонты <stat[5])                                          
        line_25_50 = result[mask]
        
        # классификатор 25-50
        
        line_25_50['надёжность'] = np.where( (line_25_50.ремонты >=stat[4]) & (line_25_50.ремонты <stat[5]), 2,3)  
        
        # от 50% до 75 % случаев
        
        mask = (result.ремонты >= stat[5]) & (result.ремонты <stat[6])                                          
        line_50_75 = result[mask]

        # классификатор 50 - 75

        line_50_75['надёжность'] = np.where( (line_50_75.ремонты >=stat[5]) & (line_50_75.ремонты <stat[6]), 3,4) 
    
        # от 75% до 100% случаев
    
        mask = (result.ремонты > stat[6])                                          
        line_75_100 = result[mask] 
    
        # классификатор 75-100
    
        line_75_100['надёжность'] = np.where( (line_75_100.ремонты >= stat[6]), 4,5) 
    
        # формирование выборки
    
        master = pd.concat([single,line_0_25,line_25_50,line_50_75,line_75_100],axis=0)
    
        # вывод в файлы 

        single.to_html('C:/Users/Battlestation/ural/надёжность_1ой_степени.html')    
        line_0_25.to_html('C:/Users/Battlestation/ural/надёжность_2ой_степени.html')    
        line_25_50.to_html('C:/Users/Battlestation/ural/надёжность_3ой_степени.html')    
        line_50_75.to_html('C:/Users/Battlestation/ural/надёжность_4ой_степени.html')
        line_75_100.to_html('C:/Users/Battlestation/ural/надёжность_5ой_степени.html') 
        
        # этап 2 формирование понятия активности
        
        # костыль одинокова анимешника которому очень плохо одному в его домашней крепост ив период изоляции. плак плак
    
        result = master.copy()  
    
        stat = result.простой.describe()
   
        # простой min
    
        mask = result.простой <=stat[3]                                          
        single_a = result[mask]
    
        # классификатор min
    
        single_a['активность'] = np.where( (single_a.простой <=stat[3]), 0,1) 
    
        # от 0 до 25% случаев 
    
        mask = (result.простой > stat[3]) & (result.простой <stat[4])                                          
        line_0_25_a = result[mask]
    
        # классификатор 0-25
    
        line_0_25_a['активность'] = np.where( (line_0_25_a.простой >=stat[3]) & (line_0_25_a.простой <stat[4]), 1,2)     
    
        # от 25% до 50 % случаев
    
        mask = (result.простой >= stat[4]) & (result.простой <stat[5])                                          
        line_25_50_a = result[mask]
    
        # классификатор 25-50
    
        line_25_50_a['активность'] = np.where( (line_25_50_a.простой>=stat[4]) & (line_25_50_a.простой <stat[5]), 2,3)  
    
        # от 50% до 75 % случаев

        mask = (result.простой >= stat[5]) & (result.простой <stat[6])                                          
        line_50_75_a = result[mask]
    
        # классификатор 50 - 75
    
        line_50_75_a['активность'] = np.where( (line_50_75_a.простой >=stat[5]) & (line_50_75_a.простой <stat[6]), 3,4) 
    
        # от 75% до 100% случаев
    
        mask = (result.простой > stat[6])                                          
        line_75_100_a = result[mask] 
    
        # классификатор 75-100
        
        line_75_100_a['активность'] = np.where( (line_75_100_a.простой >= stat[6]), 4,5) 
    
        # формирование выборки
    
        master_1 = pd.concat([single_a,line_0_25_a,line_25_50_a,line_50_75_a,line_75_100_a],axis=0)
    
        # вывод в файл
    
        single_a.to_html('C:/Users/Battlestation/ural/активность_1ой_степени.html')    
        line_0_25_a.to_html('C:/Users/Battlestation/ural/активность_2ой_степени.html')    
        line_25_50_a.to_html('C:/Users/Battlestation/ural/активность_3ой_степени.html')    
        line_50_75_a.to_html('C:/Users/Battlestation/ural/активность_4ой_степени.html')
        line_75_100_a.to_html('C:/Users/Battlestation/ural/активность_5ой_степени.html') 
        
        # этап 3ий формирование сыщности класса
        
        result = master_1.copy()
        
        # лучшее
        
        mask = (result.надёжность ==0) &  (result.активность ==0)                                       
        single_c = result[mask]
        
        # классификатор 0
        
        single_c['класс'] = np.where( (single_c.надёжность ==0) &  (single_c.активность ==0) , 0,1) 
        
        # от 0 до 25% случаев 
        
        mask = (result.надёжность ==1) &  (result.активность ==1)                                            
        line_0_25_c = result[mask]
        
        # классификатор 1
        
        line_0_25_c['класс'] = np.where( (line_0_25_c.надёжность ==1) &  (line_0_25_c.активность ==1) , 1,2)      
        
        # от 25% до 50 % случаев
    
        mask = (result.надёжность ==2) &  (result.активность ==2)                                           
        line_25_50_c = result[mask]
        
        # классификатор 2
    
        line_25_50_c['класс'] = np.where( (line_25_50_c.надёжность ==2) &  (line_25_50_c.активность ==2) , 2,3)  
        
        # от 50% до 75 % случаев
    
        mask = (result.надёжность ==3) &  (result.активность ==3)                                          
        line_50_75_c = result[mask]
        
        # классификатор 3
        
        line_50_75_c['класс'] = np.where( (line_50_75_c.надёжность ==3) &  (line_50_75_c.активность ==3) , 3,4)   
        
         # от 75% до 100% случаев
        
        mask = (result.надёжность ==4) &  (result.активность ==4)                                              
        line_75_100_c = result[mask] 
        
        # классификатор 4
        
        line_75_100_c['класс'] = np.where( (line_75_100_c.надёжность ==4) &  (line_75_100_c.активность ==4) , 4,5) 
        
        # вне класса
        
        mask = (result.надёжность ==4) &  (result.активность ==4)                                              
        line_75_100_c = result[mask] 
        
        # классификатор 4
        
        line_75_100_c['класс'] = np.where( (line_75_100_c.надёжность ==4) &  (line_75_100_c.активность ==4) , 4,5) 
       
        # формирование выборки
        
        master_2 = pd.concat([single_c,line_0_25_c,line_25_50_c,line_50_75_c,line_75_100_c],axis=0)
        
        # вывод в файл
        
        single_c.to_html('C:/Users/Battlestation/ural/машина_высшего_класса.html')    
        line_0_25_c.to_html('C:/Users/Battlestation/ural/машина_1ого_класса.html')    
        line_25_50_c.to_html('C:/Users/Battlestation/ural/машина_2ого_класса.html')    
        line_50_75_c.to_html('C:/Users/Battlestation/ural/машина_3его_класса.html')
        line_75_100_c.to_html('C:/Users/Battlestation/ural/машина_4ого_класса.html') 
        
        # этап 4ый проверяем суммы 
        
        master_3 = master_1[~master_1['_ГарНомер'].isin(master_2['_ГарНомер'])]
        
        # костыль - провеяет не потерялись ли у нас машины на перавых ддвух матерах
        
        check_0 = test[~test['_ГарНомер'].isin(master['_ГарНомер'])]
        
        check_1 = test[~test['_ГарНомер'].isin(master_1['_ГарНомер'])]

        check_2 = pd.concat([check_0,check_1],axis=0)
        
        check_2 = check_2.drop_duplicates(['_ГарНомер'], keep='last')
        
        # вывод в файл
        
        check_2.to_html('C:/Users/Battlestation/ural/машина_вне_классификатора_вообще.html')
        
        # мусор.
        
        master_3 = master_1[~master_1['_ГарНомер'].isin(master_2['_ГарНомер'])]
        
        master_3 = pd.concat([master_3,check_2],axis=0)    
        
        # вывод в файл
        
        master_3.to_html('C:/Users/Battlestation/ural/машина_вне_класса.html')
            
        # границы
        
        eva_00 = master_3.ремонты.describe()
        eva_02 = master_3.простой.describe()
        
        # пляшем вокруг стандартного распределения.
    
        master_3['класс'] = np.where((master_3.ремонты <= eva_00[2]) & (master_3.простой <= eva_02[2]), 5,6)
        
        # формирование итоговой выборки
        
        master_4 = pd.concat([master_3,master_2],axis=0)
        
        # проверка
        
        print("Исходная выборка:",len(df_n), '\n', '\n',
          "Классифицированно:", len(master_4),'\n', '\n',
          "Потери при классификации:", len(df_n)-len(master_4))    
    
        return (master,master_1,master_2,master_3,master_4),(single,line_0_25,line_25_50,line_50_75,line_75_100), (single_a,line_0_25_a,line_25_50_a,line_50_75_a,line_75_100_a),               (single_c,line_0_25_c,line_25_50_c,line_50_75_c,line_75_100_c)
    
    
    utt = ranking(my_life[0])
    
    nst = ranking(my_life[1])
    trans = ranking(my_life[2])
    nvds = ranking(my_life[3])
    cher = ranking(my_life[4])
    mtc = ranking(my_life[5])
    mtk = ranking(my_life[6])
    sam = ranking(my_life[7])
        
    return (utt,nst,trans,nvds,cher,mtc,mtk,sam)

In [10]:
def no_future():
    
    
    # внимание! в НСт найдены дубликаты объектов счисления и я решил оставить всё как есть.
    # просто запарился работать соло - без тестировщика, тесты я, идеи я, математика я, фукнционал я, логика я...
    
    # бля приплыли, теперь вместо галеры - натуральная галгофа.
    # utt отваливаеть из за str в части гар номеров, переделать через try
    
    """большая фукнция вывода, отвечает за автоматизацию, которая оперирует объектами 3х мерного массива
    
    anime[n][0][0] - перебор обществ в lonely() через N от 0 до 7
    
    [0] - utt
    [1] - nst
    [2] - trans
    [3] - nvds
    [4] - cher
    [5] - mtc
    [6] - mtk
    [7] - sam   
    
    anime[n][x][0] - перебор КАТЕГОРИЙ объектов выводва в ranking() через X от 0 до 3
    
    [n][0] - master (общая таблица)
    [n][1] - надёжность
    [n][2] - активность
    [n][3] - класс
    
    anime[n][x][z] - перебор объектов ВНУТРИ категории вывода ranking() через Z
    
    anime[n][0][0-4] - общие таблица
    anime[n][1][0-5] - таблицы надёжности
    anime[n][2][0-5] - таблицы активности
    anime[n][3][0-5] - таблицы класса
    
    также фукнция осуществляет выгрузку всех объектов в рабочую директорию, имена всегда можго изменить
    
    
    """        
        
    anime = lonely()
    
    """utt"""
    
    # utt master
    
    anime[0][0][0].to_html('C:/Users/Battlestation/ural/utt_таблица_надёжности.html')  
    anime[0][0][1].to_html('C:/Users/Battlestation/ural/utt_таблица_активности.html')  
    anime[0][0][2].to_html('C:/Users/Battlestation/ural/utt_таблица_класса.html')  
    anime[0][0][3].to_html('C:/Users/Battlestation/ural/utt_итого.html')  
    
    # utt надёжность    
    
    anime[0][1][0].to_html('C:/Users/Battlestation/ural/utt_надёжность_1ой_степени.html')
    anime[0][1][1].to_html('C:/Users/Battlestation/ural/utt_надёжность_2ой_степени.html')
    anime[0][1][2].to_html('C:/Users/Battlestation/ural/utt_надёжность_3ой_степени.html')
    anime[0][1][3].to_html('C:/Users/Battlestation/ural/utt_надёжность_4ой_степени.html')
    anime[0][1][4].to_html('C:/Users/Battlestation/ural/utt_надёжность_5ой_степени.html')
    
    # utt активность    
    
    anime[0][2][0].to_html('C:/Users/Battlestation/ural/utt_активность_1ой_степени.html')
    anime[0][2][1].to_html('C:/Users/Battlestation/ural/utt_активность_2ой_степени.html')
    anime[0][2][2].to_html('C:/Users/Battlestation/ural/utt_активность_3ой_степени.html')
    anime[0][2][3].to_html('C:/Users/Battlestation/ural/utt_активность_4ой_степени.html')
    anime[0][2][4].to_html('C:/Users/Battlestation/ural/utt_активность_5ой_степени.html')
    
    # utt класс    
    
    anime[0][3][0].to_html('C:/Users/Battlestation/ural/utt_машина_высшего_класса.html')  
    anime[0][3][1].to_html('C:/Users/Battlestation/ural/utt_машина_1ого_класса.html')  
    anime[0][3][2].to_html('C:/Users/Battlestation/ural/utt_машина_2ого_класса.html')  
    anime[0][3][3].to_html('C:/Users/Battlestation/ural/utt_машина_3ого_класса.html')  
    anime[0][3][4].to_html('C:/Users/Battlestation/ural/utt_машина_4ого_класса.html')  
    
    
    
    """nst"""
    
    # nst master
    
    anime[1][0][0].to_html('C:/Users/Battlestation/ural/nst_таблица_надёжности.html') 
    anime[1][0][1].to_html('C:/Users/Battlestation/ural/nst_таблица_активности.html') 
    anime[1][0][2].to_html('C:/Users/Battlestation/ural/nst_таблица_класса.html')  
    anime[1][0][3].to_html('C:/Users/Battlestation/ural/nst_итого.html')  
    
    # nst надёжность    
    
    anime[1][1][0].to_html('C:/Users/Battlestation/ural/nst_надёжность_1ой_степени.html')
    anime[1][1][1].to_html('C:/Users/Battlestation/ural/nst_надёжность_2ой_степени.html')
    anime[1][1][2].to_html('C:/Users/Battlestation/ural/nst_надёжность_3ой_степени.html')
    anime[1][1][3].to_html('C:/Users/Battlestation/ural/nst_надёжность_4ой_степени.html')
    anime[1][1][4].to_html('C:/Users/Battlestation/ural/nst_надёжность_5ой_степени.html')
    
    # nst активность    
    
    anime[1][2][0].to_html('C:/Users/Battlestation/ural/nst_активность_1ой_степени.html')
    anime[1][2][1].to_html('C:/Users/Battlestation/ural/nst_активность_2ой_степени.html')
    anime[1][2][2].to_html('C:/Users/Battlestation/ural/nst_активность_3ой_степени.html')
    anime[1][2][3].to_html('C:/Users/Battlestation/ural/nst_активность_4ой_степени.html')
    anime[1][2][4].to_html('C:/Users/Battlestation/ural/nst_активность_5ой_степени.html')
    
    # nst класс    
    
    anime[1][3][0].to_html('C:/Users/Battlestation/ural/nst_машина_высшего_класса.html') 
    anime[1][3][1].to_html('C:/Users/Battlestation/ural/nst_машина_1ого_класса.html')  
    anime[1][3][2].to_html('C:/Users/Battlestation/ural/nst_машина_2ого_класса.html')  
    anime[1][3][3].to_html('C:/Users/Battlestation/ural/nst_машина_3ого_класса.html')  
    anime[1][3][4].to_html('C:/Users/Battlestation/ural/nst_машина_4ого_класса.html')  
    
    """trans"""
    
    # trans master
    
    anime[2][0][0].to_html('C:/Users/Battlestation/ural/trans_таблица_надёжности.html') 
    anime[2][0][1].to_html('C:/Users/Battlestation/ural/trans_таблица_активности.html') 
    anime[2][0][2].to_html('C:/Users/Battlestation/ural/trans_таблица_класса.html')  
    anime[2][0][3].to_html('C:/Users/Battlestation/ural/trans_итого.html')  
    
    # trans надёжность    
    
    anime[2][1][0].to_html('C:/Users/Battlestation/ural/trans_надёжность_1ой_степени.html')
    anime[2][1][1].to_html('C:/Users/Battlestation/ural/trans_надёжность_2ой_степени.html')
    anime[2][1][2].to_html('C:/Users/Battlestation/ural/trans_надёжность_3ой_степени.html')
    anime[2][1][3].to_html('C:/Users/Battlestation/ural/trans_надёжность_4ой_степени.html')
    anime[2][1][4].to_html('C:/Users/Battlestation/ural/trans_надёжность_5ой_степени.html')
    
    # trans активность    
    
    anime[2][2][0].to_html('C:/Users/Battlestation/ural/trans_активность_1ой_степени.html')
    anime[2][2][1].to_html('C:/Users/Battlestation/ural/trans_активность_2ой_степени.html')
    anime[2][2][2].to_html('C:/Users/Battlestation/ural/trans_активность_3ой_степени.html')
    anime[2][2][3].to_html('C:/Users/Battlestation/ural/trans_активность_4ой_степени.html')
    anime[2][2][4].to_html('C:/Users/Battlestation/ural/trans_активность_5ой_степени.html')
    
    # trans класс    
    
    anime[2][3][0].to_html('C:/Users/Battlestation/ural/trans_машина_высшего_класса.html') 
    anime[2][3][1].to_html('C:/Users/Battlestation/ural/trans_машина_1ого_класса.html')  
    anime[2][3][2].to_html('C:/Users/Battlestation/ural/trans_машина_2ого_класса.html')  
    anime[2][3][3].to_html('C:/Users/Battlestation/ural/trans_машина_3ого_класса.html')  
    anime[2][3][4].to_html('C:/Users/Battlestation/ural/trans_машина_4ого_класса.html')  
    
    """nvds"""
    
    # nvds master
    
    anime[3][0][0].to_html('C:/Users/Battlestation/ural/nvds_таблица_надёжности.html') 
    anime[3][0][1].to_html('C:/Users/Battlestation/ural/nvds_таблица_активности.html') 
    anime[3][0][2].to_html('C:/Users/Battlestation/ural/nvds_таблица_класса.html')  
    anime[3][0][3].to_html('C:/Users/Battlestation/ural/nvds_итого.html')  
    
    # nvds надёжность    
    
    anime[3][1][0].to_html('C:/Users/Battlestation/ural/nvds_надёжность_1ой_степени.html')
    anime[3][1][1].to_html('C:/Users/Battlestation/ural/nvds_надёжность_2ой_степени.html')
    anime[3][1][2].to_html('C:/Users/Battlestation/ural/nvds_надёжность_3ой_степени.html')
    anime[3][1][3].to_html('C:/Users/Battlestation/ural/nvds_надёжность_4ой_степени.html')
    anime[3][1][4].to_html('C:/Users/Battlestation/ural/nvds_надёжность_5ой_степени.html')
    
    # nvds активность    
    
    anime[3][2][0].to_html('C:/Users/Battlestation/ural/nvds_активность_1ой_степени.html')
    anime[3][2][1].to_html('C:/Users/Battlestation/ural/nvds_активность_2ой_степени.html')
    anime[3][2][2].to_html('C:/Users/Battlestation/ural/nvds_активность_3ой_степени.html')
    anime[3][2][3].to_html('C:/Users/Battlestation/ural/nvds_активность_4ой_степени.html')
    anime[3][2][4].to_html('C:/Users/Battlestation/ural/nvds_активность_5ой_степени.html')
    
    # nvds класс    
    
    anime[3][3][0].to_html('C:/Users/Battlestation/ural/nvds_машина_высшего_класса.html') 
    anime[3][3][1].to_html('C:/Users/Battlestation/ural/nvds_машина_1ого_класса.html')  
    anime[3][3][2].to_html('C:/Users/Battlestation/ural/nvds_машина_2ого_класса.html')  
    anime[3][3][3].to_html('C:/Users/Battlestation/ural/nvds_машина_3ого_класса.html')  
    anime[3][3][4].to_html('C:/Users/Battlestation/ural/nvds_машина_4ого_класса.html')  
    
    """cher"""
    
    # cher master
    
    anime[4][0][0].to_html('C:/Users/Battlestation/ural/cher_таблица_надёжности.html') 
    anime[4][0][1].to_html('C:/Users/Battlestation/ural/cher_таблица_активности.html') 
    anime[4][0][2].to_html('C:/Users/Battlestation/ural/cher_таблица_класса.html')  
    anime[4][0][3].to_html('C:/Users/Battlestation/ural/cher_итого.html')  
    
    # cher надёжность    
    
    anime[4][1][0].to_html('C:/Users/Battlestation/ural/cher_надёжность_1ой_степени.html')
    anime[4][1][1].to_html('C:/Users/Battlestation/ural/cher_надёжность_2ой_степени.html')
    anime[4][1][2].to_html('C:/Users/Battlestation/ural/cher_надёжность_3ой_степени.html')
    anime[4][1][3].to_html('C:/Users/Battlestation/ural/cher_надёжность_4ой_степени.html')
    anime[4][1][4].to_html('C:/Users/Battlestation/ural/cher_надёжность_5ой_степени.html')
    
    # cher активность    
    
    anime[4][2][0].to_html('C:/Users/Battlestation/ural/cher_активность_1ой_степени.html')
    anime[4][2][1].to_html('C:/Users/Battlestation/ural/cher_активность_2ой_степени.html')
    anime[4][2][2].to_html('C:/Users/Battlestation/ural/cher_активность_3ой_степени.html')
    anime[4][2][3].to_html('C:/Users/Battlestation/ural/cher_активность_4ой_степени.html')
    anime[4][2][4].to_html('C:/Users/Battlestation/ural/cher_активность_5ой_степени.html')
    
    # cher класс    
    
    anime[4][3][0].to_html('C:/Users/Battlestation/ural/cher_машина_высшего_класса.html') 
    anime[4][3][1].to_html('C:/Users/Battlestation/ural/cher_машина_1ого_класса.html')  
    anime[4][3][2].to_html('C:/Users/Battlestation/ural/cher_машина_2ого_класса.html')  
    anime[4][3][3].to_html('C:/Users/Battlestation/ural/cher_машина_3ого_класса.html')  
    anime[4][3][4].to_html('C:/Users/Battlestation/ural/cher_машина_4ого_класса.html')  
    
    """mtc"""
    
    # mtc master
    
    anime[5][0][0].to_html('C:/Users/Battlestation/ural/mtc_таблица_надёжности.html') 
    anime[5][0][1].to_html('C:/Users/Battlestation/ural/mtc_таблица_активности.html') 
    anime[5][0][2].to_html('C:/Users/Battlestation/ural/mtc_таблица_класса.html')  
    anime[5][0][3].to_html('C:/Users/Battlestation/ural/mtc_итого.html')  
    
    # mtc надёжность    
    
    anime[5][1][0].to_html('C:/Users/Battlestation/ural/mtc_надёжность_1ой_степени.html')
    anime[5][1][1].to_html('C:/Users/Battlestation/ural/mtc_надёжность_2ой_степени.html')
    anime[5][1][2].to_html('C:/Users/Battlestation/ural/mtc_надёжность_3ой_степени.html')
    anime[5][1][3].to_html('C:/Users/Battlestation/ural/mtc_надёжность_4ой_степени.html')
    anime[5][1][4].to_html('C:/Users/Battlestation/ural/mtc_надёжность_5ой_степени.html')
    
    # mtc активность    
    
    anime[5][2][0].to_html('C:/Users/Battlestation/ural/mtc_активность_1ой_степени.html')
    anime[5][2][1].to_html('C:/Users/Battlestation/ural/mtc_активность_2ой_степени.html')
    anime[5][2][2].to_html('C:/Users/Battlestation/ural/mtc_активность_3ой_степени.html')
    anime[5][2][3].to_html('C:/Users/Battlestation/ural/mtc_активность_4ой_степени.html')
    anime[5][2][4].to_html('C:/Users/Battlestation/ural/mtc_активность_5ой_степени.html')
    
    # mtc класс    
    
    anime[5][3][0].to_html('C:/Users/Battlestation/ural/mtc_машина_высшего_класса.html') 
    anime[5][3][1].to_html('C:/Users/Battlestation/ural/mtc_машина_1ого_класса.html')  
    anime[5][3][2].to_html('C:/Users/Battlestation/ural/mtc_машина_2ого_класса.html')  
    anime[5][3][3].to_html('C:/Users/Battlestation/ural/mtc_машина_3ого_класса.html')  
    anime[5][3][4].to_html('C:/Users/Battlestation/ural/mtc_машина_4ого_класса.html')  
    
    """mtk"""
    
    # mtk master
    
    anime[6][0][0].to_html('C:/Users/Battlestation/ural/mtk_таблица_надёжности.html') 
    anime[6][0][1].to_html('C:/Users/Battlestation/ural/mtk_таблица_активности.html') 
    anime[6][0][2].to_html('C:/Users/Battlestation/ural/mtk_таблица_класса.html')  
    anime[6][0][3].to_html('C:/Users/Battlestation/ural/mtk_итого.html')  
    
    # mtk надёжность    
    
    anime[6][1][0].to_html('C:/Users/Battlestation/ural/mtk_надёжность_1ой_степени.html')
    anime[6][1][1].to_html('C:/Users/Battlestation/ural/mtk_надёжность_2ой_степени.html')
    anime[6][1][2].to_html('C:/Users/Battlestation/ural/mtk_надёжность_3ой_степени.html')
    anime[6][1][3].to_html('C:/Users/Battlestation/ural/mtk_надёжность_4ой_степени.html')
    anime[6][1][4].to_html('C:/Users/Battlestation/ural/mtk_надёжность_5ой_степени.html')
    
    # mtk активность    
    
    anime[6][2][0].to_html('C:/Users/Battlestation/ural/mtk_активность_1ой_степени.html')
    anime[6][2][1].to_html('C:/Users/Battlestation/ural/mtk_активность_2ой_степени.html')
    anime[6][2][2].to_html('C:/Users/Battlestation/ural/mtk_активность_3ой_степени.html')
    anime[6][2][3].to_html('C:/Users/Battlestation/ural/mtk_активность_4ой_степени.html')
    anime[6][2][4].to_html('C:/Users/Battlestation/ural/mtk_активность_5ой_степени.html')
    
    # mtk класс    
    
    anime[6][3][0].to_html('C:/Users/Battlestation/ural/mtk_машина_высшего_класса.html') 
    anime[6][3][1].to_html('C:/Users/Battlestation/ural/mtk_машина_1ого_класса.html')  
    anime[6][3][2].to_html('C:/Users/Battlestation/ural/mtk_машина_2ого_класса.html')  
    anime[6][3][3].to_html('C:/Users/Battlestation/ural/mtk_машина_3ого_класса.html')  
    anime[6][3][4].to_html('C:/Users/Battlestation/ural/mtk_машина_4ого_класса.html')  
    
    """sam"""
    
    # sam master
    
    anime[7][0][0].to_html('C:/Users/Battlestation/ural/sam_таблица_надёжности.html') 
    anime[7][0][1].to_html('C:/Users/Battlestation/ural/sam_таблица_активности.html') 
    anime[7][0][2].to_html('C:/Users/Battlestation/ural/sam_таблица_класса.html')  
    anime[7][0][3].to_html('C:/Users/Battlestation/ural/sam_итого.html')  
    
    # sam надёжность    
    
    anime[7][1][0].to_html('C:/Users/Battlestation/ural/sam_надёжность_1ой_степени.html')
    anime[7][1][1].to_html('C:/Users/Battlestation/ural/sam_надёжность_2ой_степени.html')
    anime[7][1][2].to_html('C:/Users/Battlestation/ural/sam_надёжность_3ой_степени.html')
    anime[7][1][3].to_html('C:/Users/Battlestation/ural/sam_надёжность_4ой_степени.html')
    anime[7][1][4].to_html('C:/Users/Battlestation/ural/sam_надёжность_5ой_степени.html')
    
    # sam активность    
    
    anime[7][2][0].to_html('C:/Users/Battlestation/ural/sam_активность_1ой_степени.html')
    anime[7][2][1].to_html('C:/Users/Battlestation/ural/sam_активность_2ой_степени.html')
    anime[7][2][2].to_html('C:/Users/Battlestation/ural/sam_активность_3ой_степени.html')
    anime[7][2][3].to_html('C:/Users/Battlestation/ural/sam_активность_4ой_степени.html')
    anime[7][2][4].to_html('C:/Users/Battlestation/ural/sam_активность_5ой_степени.html')
    
    # sam класс    
    
    anime[7][3][0].to_html('C:/Users/Battlestation/ural/sam_машина_высшего_класса.html') 
    anime[7][3][1].to_html('C:/Users/Battlestation/ural/sam_машина_1ого_класса.html')  
    anime[7][3][2].to_html('C:/Users/Battlestation/ural/sam_машина_2ого_класса.html')  
    anime[7][3][3].to_html('C:/Users/Battlestation/ural/sam_машина_3ого_класса.html')  
    anime[7][3][4].to_html('C:/Users/Battlestation/ural/sam_машина_4ого_класса.html')  
    
    
    
    return anime

In [11]:
def simple(df_n,rout,n,warp):  
    
    # откликаеться на test = simple(df_n[0][0],rout[0],0)
     
    # [df_n] - датафрейм который мы передаём в фукнцию, передавать надо [0][0], [1][0], [2][0] и т.д
    # rout - мартшрут к записям о новых машинах 
    # [n] - номер варпа
    
    """
    Математика

    .mean() - ариф.сред
    .median() - медиана

    Число_операций == .count()
    Среднее_время_ремонта == .mean()
    Стандартное_отклонение == .std()
    Максимальное_время_ремонта == .max()

    Старта цикла == [цикл / число_машин] > 1 < 2
    Устойчивый цикл == [цикл/ число_машин] > 2

    Логика

    Постоянство_признак_мастерства ==  .mean() > .strd()

    Лень_матушка == .mean() + .std() > .max()

    Результат

    Оценка_процесса == .Постоянство_признак_мастерства(Проблема) ~ Лень_матушка(Проблема)
    Критические_проблемы == Оценка_процесса(процесс>2)

    """
    for pony in warp[n]: # расширить вывод. сделано

        # подаём уникальное имя машины с гаражным номером в генератор, получаем датафрейм в отношении одной машины

        name = pony

        df = df_n[df_n['Unic_gar']==name] 

        # читаем файл с закупками новых машин по обществу

        car_in = pd.read_excel(rout, index_col=0,
                        dtype={'Name': str, 'Value': float})

        # храним гаражный номер машины для сводной таблицы.

        number = df['_ГарНомер'].values        

        # сбрасываем unnamed в файле и переименовываем инв. номер в признак _gar

        car_in = car_in.drop(car_in.columns[car_in.columns.str.contains('unnamed', case = False)], axis = 1)
            
        # новые файлы иногда поступаю с дубликами записей - отпраялем дубликаты в варп
            
        car_in.rename(columns={car_in.columns[0]:'_gar'},inplace=True)
        car_in.rename(columns={car_in.columns[1]:'VIN'},inplace=True)
        car_in.rename(columns={car_in.columns[2]:'год_выпуска'},inplace=True)
        car_in.rename(columns={car_in.columns[4]:'Поставщик'},inplace=True)
        car_in.rename(columns={car_in.columns[5]:"Цена без НДС"}, inplace=True)     
        
        car_in = car_in.drop_duplicates(['_gar'], keep='last')
        
        # иногда в ремонтных листах ГарНомер записан через жопу , приводим к int64
        
        # отключено  в UTT гар.номера пишут с буквой на конце
        
        if n >0:
            df['_ГарНомер'] = df['_ГарНомер'].astype('float')
            df['_ГарНомер'] = df['_ГарНомер'].astype('int64')            
            
        # иногда год выпуска записан через жопу
            
        try:
            car_in = car_in.sort_values('год_выпуска',ascending=False)
        except KeyError:
            car_in = car_in.rename(columns={'Дата выпуска ': 'год_выпуска'})
            car_in['год_выпуска'] = car_in['год_выпуска'].astype('Datetime64')
        
        # проверяем датафрейм из генератора на принадлежность к списку новой техн
        
        eva_04 = df.loc[df['_ГарНомер'].isin(car_in._gar.values)]
                                             
        # в случае, если имеем дело с новой машиной возвращаем гаражный номер, vin, год выпуска, цены, поставщщ
        
        if len(eva_04) >=1:
            car_list = car_in.loc[car_in['_gar'].isin(eva_04._ГарНомер.values)]
            car_list_vin = car_list.VIN.values # 19.03.2020 перестал оборачивать в лист дома
            car_list_year = list(car_list['год_выпуска'].values)
            car_list_price = list(car_list['Цена без НДС'].values)
            car_list_registration = list(car_list['Дата принятия к учету'].values)
            car_list_seller = list(car_list.Поставщик.values)
        else:
            car_list = None
            car_list_vin = None
            car_list_year = None
            car_list_price = None
            car_list_registration = None
            car_list_seller = None
            
        # принт выведен в vision 

        print('Это новая машина, vin: ', car_list_vin," ", 
                      'год_выпуска: ', car_list_year,'\n','\n',

                      'цена: ', car_list_price,
                      "поставщик: ",  car_list_seller,'\n',
                      'Дата принятия к учёту: ', car_list_registration, sep='')

        # в случае, если датафрейм с машиной содержит более 2х записей - принимаем его в работу, это 2 поломки в год 
        # в противном случае выводим предупреждение, else в самом низу няша, если else значит ломается раз в год

        """size - считает каждую ремонтную операцию уникальной, как правило size больше чем кол-во рем.листов
            по причине того, что в одном ремонтном листе может быть записано несколько различных ремонтных операций"""

        size = df.Операции.nunique()      

        """true_size - считает каждый ремонтный лист уникальным эпизодом ремонта, как правило true_size меньше size"""

        true_size = len(df._Номер.unique())

        # float_size считает количество операций на один эпизод

        try:
            float_size = size/len(df)
        except ZeroDivisionError:
            float_size = 0
            
        float_size = float('{:.3f}'.format(float_size))

        """math - предназначен для поиска интервалов между эпизодами,
            math - считает каждый эпизод уникальным, вне зависимости от кол-ва ремонтных операций в эпизоде
            math - оперирует метрикой цикличности range - дистанцией в днях от первого рем.листа до второго и т.д"""

        # второй бэкап, чтобы не трогать исходную таблицу с машиной до преобразования

        me = df.copy()

        # нужно добавить в me фильтр только по уникальным операциям - иначе range будет лишен смысла в выводе.

        me = me.drop_duplicates(['_Номер'], keep='last')

        #  сортировка дат в календарном порядке, сначала первый лист в году потом последний   
        
        """ 
        18.03.2020 отключил дома
        
        me['_Дата'] = me['_Дата'].astype('datetime64')
        
        13.04 неожиданным образом это вызвало конфликт при работе с нст!
        
        возвращаею обратно, запускаю тест и это сразу вызвало исключение nontype obj is not interable
        
        раз в лоб не получаеться - убираю, пробую через трай
        
        """
        me = me.sort_values(by="_Дата", ascending = True)

        # формирование признака range - дистанция, в днях между ремонтными листами начиная с первого
        
        try:
            me['range'] = me._Дата.diff().dt.days
        except TypeError:
            me['_Дата'] = me['_Дата'].astype('datetime64')
            me['range'] = me._Дата.diff().dt.days    
    
        """
        уехало в трай
        
        me['range'] = me._Дата.diff().dt.days
        """

        # нулевая дистанция нас не интересует, поскольку быстрые ремонты означают хорошую работу РММ.

        mask = me.range>0                                            
        me_1 = me[mask]

        # загатовка под логистическую регрессию, без нулей в выборке, считать нули не всегда полезно.

        me_log = me_1.copy()
        me_log = me_log.drop(['Машины','Unic_gar'],axis=1)  

        # сбрасываем качественные признаки для расчёта математических

        math = me.drop(['Машины','Unic_gar'],axis=1)

        # считаем стандартное распределение, арф.среднее и медиану по показателям признака range

        math['std_m'] = math.range.std()
        math['mean_m'] = math.range.mean()
        math['med_m'] = math.range.median()

        """маска подсчёта случаев сверх быстрого ремонта в ноль дней - идёт в принт, очень интересная метрика
               поскольку мы уже сбросили все неуникальные ремонтные листы, то 0 дней будет означать новый рем.лист
               при этом количество листов, которые мы подсчитываем будет на 1 меньше т.к первый лист имеет значение
               NaN по причине того, что его несчего считать."""

        mask = math.range==0
        math_zero = math[mask]
        fast = len(math_zero)                                       

        # маска подсчёта случаев ремонта от 0 до 10 дней - идёт в принт (типа плохие)

        mask = (math.range>0) & (math.range <=10)
        math_range = math[mask]
        bad_manga = len(math_range) 

        # маска подсчёта случаев ремонта от 10 дней - идёт в принт (условно хорошие)

        mask = (math.range>10)                                       
        math_outside_10 = math[mask]
        normal = len(math_outside_10)  

        # маска подсчёта ремонтов чаще стандартного отклонения
                                             
        mask = (math.range>0) &( math.range<= math.std_m)           
        math_range_std = math[mask]
        bad_std = len(math_range_std)

        # маска подсчёта ремонтов чаще средне.арифмитического

        mask = (math.range>0) &( math.range<= math.mean_m)          
        math_range_mean = math[mask]
        bad_mean = len(math_range_mean)

        # ремонты чаще IQL, при этом IQL может не посчитать

        mask = (math.range>0) &( math.range<= math.med_m)           
        math_range_iql = math[mask]
        bad_iql = len(math_range_iql)

        # если в IQL полезной инфы нет, признаём это истиной  

        iql = None

        if math.med_m.unique() <1:
            iql = True
            print('Внимание! В интервалах постановки на ремонт отсутсвует явная зависимость.','\n') 

        # вычисляе метрику долей - ненормальная периодика в %

        try:        
            bad_val_percent = 100 * bad_manga / normal
        except ZeroDivisionError: 
            bad_val_percent = 0

        float_negative = float('{:.3f}'.format(bad_val_percent))

        """Верхняя граница это попытка определят интервал эипзодов ремонта с самым большим допустимым допуском
           Макмиальное среднее - показания Q2 и арф.среднего с наибольшими показателями, как следствие такого
           разнобоя в цифрах, в отношении которого единый подход определения усреднённых значений не вариант

           Наибольшим допустимым интервалом ремонта считается сумма стандартного отклонения и занчения макс.среднего
           Если интервал между ремонтными эпизодами больше Верхней границы - всё нормально, если меньше - проблема. 
           """

        try:
            math['Максимальное_среднее'] = math[['med_m','mean_m']].max(axis=1)

            math['Верхняя_граница'] =  math.range > (math.Максимальное_среднее + math.std_m) 
            math.Верхняя_граница =  math.Верхняя_граница.astype('bool')

            anime = {True:'Нормально', False: 'Проблема'}
            math.Верхняя_граница = math.Верхняя_граница.map(anime)

            dark_list = math[math['Верхняя_граница'].str.contains('Проблема', na=False)]
            dark_list_int = len(dark_list)

            good_manga = math[math['Верхняя_граница'].str.contains('Нормально',na= False)]
            good_manga = len(good_manga)
        except KeyError:
            result['Максимальное_среднее'] = np.nan
            result['Верхняя_граница'] = np.nan   

        """блок подсчёта истинного простоя

            unique описывает ремонтные листы - основная идея - подсчитать значения по уникальным эпизода ремонта
            для этого мы сбрасываем все дубликаты ремонтных листов, которые существуют в исходном df 
            да, в исходном df могут быть дубликаты, поскольку на разные операции пишут один лист

            и продолжат писать дальше, мы от этого не уйдём и я уже сломал себе голову на постобработке двух
            сводных таблиц, этой и второй, которая считает цикличность уникальных операций.

            это очень грустаня история пропалку с двумя разными концами, в конце которой получается лампосин
            если вы видели предыдущие вресии алгоритма - вы знаете чем это кончилось и почему я решил писать новый

            оперировать списком множества операций в отношение уникального ремонтного листа тоже не вариант.

            короче, unique это истинны -  а не мнимые цифр, поэтому мы работаем с ней.

            delta == ['ДатаКонца'] - ['ДатаНач'] т.е времени, которое потребовалось на ремонтные работы по листу

            в индексе все уникальные случаи постановки машины на ремонт               
                """

        unique = df.drop_duplicates(['_Номер'], keep='last')
        
        try:

            pvt_mean= unique.pivot_table(index = ['_Номер'], columns = ['Unic_gar'],values = 'delta', aggfunc=lambda x: x.mean())
            pvt_mean.rename(columns={name:"Среднее_время_ремонта"},inplace = True)                            

            pvt_max = unique.pivot_table(index = ['_Номер'], columns = ['Unic_gar'],values = 'delta', aggfunc=lambda x: x.max())
            pvt_max.rename(columns={name:"Максимальное_время_ремонта"},inplace = True)

            pvt_count = unique.pivot_table(index=['_Номер'], columns = ['Unic_gar'], values ='delta',aggfunc=lambda x: x.count())
            pvt_count.rename(columns={name:"Число_операций"},inplace= True)

            pvt_min = unique.pivot_table(index = ['_Номер'], columns = ['Unic_gar'],values = 'delta', aggfunc=lambda x:x.min())
            pvt_min.rename(columns={name:"Минимальное_время_ремонта"},inplace = True)

            pvt_med = unique.pivot_table(index = ['_Номер'], columns = ['Unic_gar'],values = 'delta', aggfunc=lambda x:x.median())
            pvt_med.rename(columns={name:"Хитрое_время_ремонта"},inplace = True)

            pvt_std = unique.pivot_table(index = ['_Номер'], columns = ['Unic_gar'], values = 'delta', aggfunc=lambda x: x.std(),dropna =False)
            pvt_std.rename(columns={name:"Стандартное_отклонение"},inplace = True) 

            u_result = pd.concat([pvt_mean,pvt_med, pvt_std, pvt_max,pvt_min,pvt_count],axis=1)
            
        except AttributeError:
            print('нани',name)   

        # витрина содержжит цифры только по уникальным ремонтным операциям, посчитаем время простоя в год

        unique_time = unique.delta.sum()      

        # bad_time вынесен в vision

        bad_time = None

        if (unique_time > pd.Timedelta(30, unit='d')) & (unique_time <pd.Timedelta(90, unit='d')):
            bad_time = 'Машина простояла в ремонте больше месяца'
            print('Внимание! Машина простояла в ремонте больше месяца')
        elif (unique_time > pd.Timedelta(90, unit='d')) & (unique_time < pd.Timedelta(180, unit='d')):
            bad_time = 'Машина простояла в ремонте больше квартала'
            print('Внимание! Машина простояла в ремонте больше квартала')
        elif (unique_time > pd.Timedelta(180, unit='d')) & (unique_time < pd.Timedelta(270, unit='d')):
            bad_time = 'Машина простояла в ремонте больше полугодия'
            print('Внимание! Машина простояла в ремонте больше полугодия')
        elif unique_time > pd.Timedelta(270, unit='d'):
            bad_time = 'Машина простояла в ремонте почти год'
            print('Внимание! Машина простояла в ремонте почти год') 

        """блок циклов уникальных операций - очень противоречивая вещь, которую я в итоге решил оставить
                проблема в следующем: индикатор delta является отсылкой к ремонтному листу с несколькими операциями

                таким образом установить -сколько времени было затраченно на одну операцию в рамках листа с несколькими               
                невозможно, но мы можем подсчитать абстрактные цифры - цифры ремонтного эпизода, в которых есть операция.

                """

        # абстракция №1: индекс - виды операций, цирфы - длительность рем.листа по операции

        pvt_mean= df.pivot_table(index = ['Операции'], columns = ['Unic_gar'],values = 'delta', aggfunc=lambda x: x.mean())
        pvt_mean.rename(columns={name:"Среднее_время_ремонта"},inplace = True)                            

        pvt_max = df.pivot_table(index = ['Операции'], columns = ['Unic_gar'],values = 'delta', aggfunc=lambda x: x.max())
        pvt_max.rename(columns={name:"Максимальное_время_ремонта"},inplace = True)

        pvt_count = df.pivot_table(index=['Операции'], columns = ['Unic_gar'], values ='delta',aggfunc=lambda x: x.count())
        pvt_count.rename(columns={name:"Число_операций"},inplace= True)

        pvt_min = df.pivot_table(index = ['Операции'], columns = ['Unic_gar'],values = 'delta', aggfunc=lambda x:x.min())
        pvt_min.rename(columns={name:"Минимальное_время_ремонта"},inplace = True)

        pvt_med = df.pivot_table(index = ['Операции'], columns = ['Unic_gar'],values = 'delta', aggfunc=lambda x:x.median())
        pvt_med.rename(columns={name:"Хитрое_время_ремонта"},inplace = True)

        pvt_std = df.pivot_table(index = ['Операции'], columns = ['Unic_gar'], values = 'delta', aggfunc=lambda x: x.std(),dropna =False)
        pvt_std.rename(columns={name:"Стандартное_отклонение"},inplace = True) 

        """result содержит мнимые цифры!

                result предназначен для поиска циклов и определения ошибок в циклах ремонта техники
                result не может посчитать время простоя по эпизоду, потому что в эпизоде несколько операций 

                result описывает длительность ремонтных операций считая их уникальными событиями и складывает в циклы
                циклы позволяют оценивать способность РММ проводить однотипные операции по конкретной машине по delta

                delta == ['ДатаКонца'] - ['ДатаНач'] т.е времени, которое потребовалось на ремонтные работы по листу"""

        result = pd.concat([pvt_mean,pvt_med, pvt_std, pvt_max,pvt_min,pvt_count],axis=1)

        try:

            """перехват исключений написан на более ранню версию алгоритма, работает - не трогай!                   

                    перехват нулевых значений стандартного отклонения в result,
                    стандартное отклонение в result может показать нули
                    это исключение необходимо обрабатывать"""

            # проверяем наличе признака Стандартное_Отклонение просто вызвав его (всё простое лучше сложного)

            result['Стандартное_отклонение']

            # теория 1: арф.среднее время ремонта не должно превышать стандатное отклонение в отношении операции

            result['Постоянство_признак_мастерства'] = result.Среднее_время_ремонта > result.Стандартное_отклонение            
            result.Постоянство_признак_мастерства =  result.Постоянство_признак_мастерства.astype('bool')

            # в случае, если утверждение верно - проблем нет, в случае если утверждение ложно - проблемы есть

            anime = {True:'Нормально', False: 'Проблема'}               
            result.Постоянство_признак_мастерства = result.Постоянство_признак_мастерства.map(anime)

            # теория 2: сумма арф.среднего времени ремонта и стандартного отклонения не должна превышать максимум по времени

            result['Лень_матушка'] = result.Среднее_время_ремонта + result.Стандартное_отклонение > result.Максимальное_время_ремонта
            result.Лень_матушка =  result.Лень_матушка.astype('bool')

            # в случае, если утверждение верно - пробелм нет, в случае если утверждение ложно - проблемы есть

            anime = {True:'Проблема', False: 'Нормально'}
            result.Лень_матушка = result.Лень_матушка.map(anime) 

            # теория 3: если в обоих случаях отмечаются проеблым, поставнока процесса ремонта в РММ считаеться плохой

            result['Оценка_процесса'] = (result['Постоянство_признак_мастерства'].str.contains('Проблема',na=False)&
                                                   result['Лень_матушка'].str.contains('Проблема',na=False))

            result.Оценка_процесса =  result.Оценка_процесса.astype('bool')

            anime = {True:'Проблема', False: 'Нормально'}
            result.Оценка_процесса = result.Оценка_процесса.map(anime)      

            # возможны различные случаи оценки, для того чтобы алгоритм сработал нужно предусмотреть их все. 

        except KeyError:
            result['Стандартное_отклонение'] = np.nan
            result['Оценка_процесса'] = np.nan
        except AttributeError:
            result['Постоянство_признак_мастерства'] = np.nan

            # характер данных может приводить к ошибке среднего времени, её тоже необходимо перехватывать

        try:               
            result = result.sort_values(by='Среднее_время_ремонта', ascending=False)
        except KeyError:
            print(pony,' ошибка среднего времени')       

        # запоминаем длинну result и eva_4

        music = len(result)
        fear_of_the_dark = len(eva_04)

        # создаём признак Усредённое среднее - содержит минимальные значение времени ремонта по арф.сред и Q2 

        try:
            result['Усредненное_среднее'] = result[['Среднее_время_ремонта','Хитрое_время_ремонта']].min(axis=1)

        # признак Нижняя граница соотвествует принципу: минимальное время ремонта эквивалентно арф.сред и Q2

            result['Нижняя_граница'] = result.Усредненное_среднее <= result.Минимальное_время_ремонта 
            result.Нижняя_граница =  result.Нижняя_граница.astype('bool')

            anime = {True:'Нормально', False: 'Проблема'}
            result.Нижняя_граница = result.Нижняя_граница.map(anime)

        # ввод признака Строгая_оценка - совпадение всех индикаторов проблема в матрице и наоборот

            result['Строгая_оценка'] = (result['Оценка_процесса'].str.contains('Проблема',na=False)&
                                                   result['Нижняя_граница'].str.contains('Проблема',na=False))

            result.Строгая_оценка =  result.Строгая_оценка.astype('bool')

            anime = {True:'Проблема', False: 'Нормально'}
            result.Строгая_оценка = result.Строгая_оценка.map(anime) 

        # в случае, если признак Усреднённое среднее не считаеться - пишем nan

        except KeyError:
            result['Усредненное_среднее'] = np.nan

        # pro_error - оценку качества работы РММ, 0 - хорошо, 1 - ошибка, 2 - две ошибки   

        pro_error = None

        try:        
            error = result.Строгая_оценка.value_counts().Проблема
            error_list = result[result['Строгая_оценка'] =="Проблема"].index
            error_list = [x for x in error_list]
            pro_error = len(error_list)
        except AttributeError:
            print("Внимание! В оценке процесса критических проблем не обнаружено")
            error = 'проблем нет'
            error_list = 'проблем нет'

        # число операций

        opr = len(result[result['Число_операций'] ==1])
        opr_repeat = len(result[result['Число_операций'] >1])

        # влияет на вывод тетрадки - показывает служебные витрины сразу в том размере, в каком они существуют.

        try:                
            pd.set_option('display.max_rows',fear_of_the_dark)
        except NameError:
            pd.set_option('display.max_rows',music)

        """добавление цикличности в качестве признака в базовую таблицу, таблицу цикличности храним в base"""    

        base = result.loc[result['Число_операций'] >1]

        # цикл это операция, которая повторилсь хотя бы один раз, устойчивый цикл - повторился более 3х раз

        result['цикл'] = result.Число_операций / 2                 
        result['старт_цикла'] = (result.цикл <2) & (result.цикл >= 1)          
        result['устойчивый_цикл'] = result.цикл >= 2 

        # подсчёт всех циклов

        toga = result[result['Число_операций'] > 1]        
        fix_count = toga.Число_операций.sum() 

        # подсчёт новых циклов

        kirito = result[result['старт_цикла'] == True]        
        new_fix = kirito.старт_цикла.sum() 

        # подсчёт устойчивых циклов

        darkness = result[result['устойчивый_цикл'] == True]        
        old_fix = darkness.устойчивый_цикл.sum()

        # подсчёт ремонтных операций

        count_times = result.Число_операций.values.sum()

        # подсчёт всех операций в циклах

        all_fix = new_fix + old_fix

        """добавление оценки качества циклов и списка ошибок, except AttributeError перехватывает исключение нулей"""

        # формирование списка ошибок, по индексу из общей витрины, перехватывает ислючение в случае если ошибок нет

        save_record = None

        # save_record - проблемы в циклах.

        try:        
            error = result.Оценка_процесса.value_counts().Проблема
            error_list = result[result['Оценка_процесса'] =="Проблема"].index
            error_list = [x for x in error_list]
            save_record = len(error_list)
        except AttributeError:
            print("Внимание! В оценке процесса проблем не обнаружено")
            error = 'проблем нет'
            error_list = 'проблем нет'

        # формирование спика критических ошибок по циклам из общей витрины, также перехватывает исключение нулей 

        save_list = None

        # save_list - проблемы в стабильных циклах 

        try:
            crit_error = result[result['Оценка_процесса'] =='Проблема']
            crit_error_list = crit_error[crit_error['Число_операций'] >2]
            save_list = len(crit_error_list)
        except (KeyError, TypeError) as e:
            crit_error = np.nan
            crit_error_list = np.nan   

        # проверка списка критических ошибок и оповещение пользователя об их отсутствии в выборке        

        try:        
            if len(crit_error_list)>0:
                crit_error_list = crit_error_list.index                
            else:
                crit_error_list = None
        except TypeError:
            print('всё хорошо, они просто не очень редко ломаются, но сводку посчитало')

        # циклы - осторожно, ломает голову в попытке внести правки или дополнения

        old_base = result[result['устойчивый_цикл'] == True] 
        old_base_int = old_base.Число_операций
        count_fix = old_base_int.values.sum()

        new_base = result[result['старт_цикла'] == True]
        new_base_int = new_base.Число_операций        
        count_fix_new = new_base_int.values.sum()

        # доля цикличных операций

        mis_val_percent = 100 * all_fix / len(df)
        float_percent = float('{:.3f}'.format(mis_val_percent))

        # индикатор цикличности

        face = None

        if float_percent > 20:
            face = 'Внимание! Цикличная машина - готовьте запчасти'
            print("Внимание! Цикличная машина - готовьте запчасти")
        elif (float_percent < 20) & (float_percent > 5):
            face = "Внимание! Обратите внимание на циклы операций"
            print("Внимание! Обратите внимание на циклы операций")
        elif (float_percent < 5):
            face = "Внимание! Доля циклов мала, считает плохо"
            print('\n',"Внимание! Доля циклов мала, считает плохо",sep="")

        # следим за ремонтам ДВС

        """очень неудачная констркция - по строке ДВС как правило идёт куча долива масла, это нужно обдумать в октябре"""

        dvs= None # привет шапр, да я поустилс до того что стал явно объявлятб переменные

        if len(base) >1:      
            if len(toga.iloc[toga.index.str.contains('РЕМОНТ_ДВС',case=False, regex=True)]) >1:
                dvs = True
                print('Внимание! Цикличный ремонт двигателя')

        # результрующая всего и вся!

        """Ищенко считает, что машина не должна чиниться чаще чем раз в 10 дней, 
               но алгоритм уже умеет подбираться эту метрику самостоятельно, поэтому:
               - установим правило, если модель Ищенко больше модели алгоритма - alarm!
               """

        alarm = None

        if math.Максимальное_среднее.values[0] < 10:
            alarm = True
            print('\n',"AlaRm: ТС привыкли чинить чаще 10 дней", sep='')
        else:            
            pass            

        # 05_09_2019 последний штрих - метрика позитивной переодичности        

        try:        
            math_val_percent = 100 * good_manga / dark_list_int 
        except ZeroDivisionError: 
            math_val_percent = 0  

        # костыль 

        # это нужно проверить!!!

        series = {'ТС': [pony],

                      'Гаражный номер' :[number[0]],

                      'Кол-во постановок на ремонт': [true_size],                  # кол-во рем листов                  
                      "Кол-во выполненых ремонтных операций" : [len(df)],          # кол-во операций по листам                  
                      "Кол-во уникальных типов ремонтных операций": [size],        # кол-во не повторящихся операций              

                      "Доля уникальных ремонтных операций" : [float_size],         # отношение уникальных операций ко всем
                      "Кол-во циклов" :  [all_fix],                                # кол-во типов опр. которые повторяються                

                      'Доля негативной периодичности операций': [float_negative],  # отношение постановок в пр.Ищенко ко всем
                      "Простой по всем ремонтным событиям" :[unique_time],         # сумма простоя по всем ремонтным листам

                      "Число устойчивых циклов" : [old_fix],                       # сумма циклов с большой долей повторения
                      "Число потенциальных циклов" : [new_fix],                    # сумма циклов с небольшой долей повторения

                      "Операции в устойчивых циклах" : [count_fix],                # кол-во операций в устойчивых циклах                  
                      "Операции в новых циклах" : [count_fix_new],                 # кол-во операций в новых циклах

                      "Количество постановок на ремонт чаще 10 дней" : [bad_manga],          # число постановок в пределе Ищенко
                      "Количество постановок на ремонт с интервалом 0 дней" : [fast],        # число постановок с интервалом 0
                      "Количество постановок на ремонт с нормальном интервалом" : [normal],  # число постановок за пределом

                      "Номера ремонтных листов ниже предела Ищенко": [list(math_range._Номер.values)],                   
                      "Вероятно значение математического предела": [float('{:.3f}'.format(math.Максимальное_среднее.values[0]))], #1

                      "Эпизоды ниже этой границы": [dark_list_int],    # кол-во постановок ниже математического предела
                      "Эпизоды выше этой границы": [good_manga],       # кол-во постановок выще математического предела

                      "Номера ремонтных листов ниже математического предела" : [list(dark_list._Номер.values)],
                      "Доля позитивной математической периодичности" :[float('{:.3f}'.format(math_val_percent))], 
                      "Номера ремонтных листов с периодичностью в 0 дней" : [list(math_zero._Номер.values)],

                      "Это новая машина, vin":[car_list_vin] , 
                      "год_выпуска": [car_list_year], 
                      "цена":[car_list_price], 
                      
                      "поставщик": [car_list_seller],                   

                      "Внимание! Цикличный ремонт двигателя": [dvs],                 #  не доработано
                      "AlaRm: ТС привыкли чинить чаще 10 дней" :[alarm],             #  предупреждение

                      "Оценка ценности IQL":[iql],                                   #  только для математиков
                      "Оценка цикличности ТС": [face],                               #  3 письменные градации степени циклов

                      'Кол-во серьёзных проблем в РММ': [pro_error],    # result -строгая оценка работы РММ
                      'Описание серьёзных проблем': [error_list],       # список опреаций, которые попали в строгий неуд в РММ

                      'Кол-во проблем в РММ':[save_record],  # не строгая, но плохая оценка работы РММ в циклах
                      'Описание проблем':[error_list],       # список операций, которые попали под трояк

                      'Количество проблем в устойчивых циклах': [save_list],    # проблемы в стабильных циклах
                      "Описание проблем в устойчивых циклах":[crit_error_list], # описание проблем в стабильных циклах

                      'Характер простоя':[bad_time],                            # писмьенное предупреждение, 4 вида

                      }

        tablet = pd.DataFrame(data=series)       

        tablet.style.set_properties(subset=['Описание проблем в РММ'], **{'width': '300px'})

        print('\n',                  
                    "Сводка: ",'\n','\n', pony, '\n','\n',

                    "Кол-во постановок на ремонт: ",true_size,'\n',
                    "Кол-во выполненых ремонтных операций: ", len(df),'\n',  

                    "Кол-во уникальных типов ремонтных операций : ", size,'\n',
                    "Кол-во уникальных типов ремонтных операций на эпизод: ", float_size,'\n','\n',

                    "Кол-во повторяемых типов ремонтных операций: ",  all_fix, '\n',
                    'Доля цикличности в ремонтных операциях: ', float_percent,'%', '\n',
                    'Доля негативной периодичности операций: ', float_negative , '%','\n','\n',  

                    "Общий Простой", '\n','\n',

                    "Простой по всем ремонтным событиям: ",unique_time, '\n', "\n",               

                    "Циклы",'\n','\n',

                    "Общее число циклов: ", all_fix, '\n',
                    "Число устойчивых циклов: ", old_fix, "\n",
                    "Число потенциальных циклов: ", new_fix , "\n","\n",

                    "Операции в устойчивых циклах: ", count_fix,"\n","\n", old_base_int, '\n', "\n",

                    "Операции в новых циклах: ", count_fix_new,"\n","\n", new_base_int, '\n', "\n", 

                    "Периодичность эпизодов ремонта",'\n','\n',

                    "Количество постановок на ремонт чаще 10 дней: ", bad_manga, '\n',
                    "Количество постановок на ремонт с интервалом 0 дней: ", fast,'\n',
                    "Количество постановок на ремонт с нормальном интервалом: " , normal , '\n',
                    "Номера ремонтных листов по операциям ниже границы Льва Леонидовича: ", list(math_range._Номер.values), '\n','\n',

                    "Математические границы периодичности" , '\n','\n',

                    "Наибольшая грацица: ", float('{:.3f}'.format(math.Максимальное_среднее.values[0])),'\n',
                    "Эпизоды ниже этой границы: ", dark_list_int,'\n',
                    "Эпизоды выше этой границы: ", good_manga, '\n',

                    "Номера ремонтных листов по эпизодам постановки на ремонт ниже математической границы: " ,'\n',
                    list(dark_list._Номер.values), '\n','\n',

                    "Доля позитивной математической периодичности: ",float('{:.3f}'.format(math_val_percent)),'%' '\n','\n',

                    "Номера ремонтных листов с периодичностью в 0 дней: ", list(math_zero._Номер.values), '\n',

                    '__________','\n',

                     sep="")  

        return tablet

In [12]:
def coub_01(x,n,w): 
    
    """исправленое 01.04.2020 - не ехелаа без передачи аргумент влоб
    
       c coub() внутри считаеться меделлно - но другого пути пока не вижу нет"""

    one = pd.DataFrame() # кеш
    
    df_n,warp,rout = coub()
        
    def call(one,df_n,warp,rout):    # передача кеша, указатель на x - датафрейм, n - путь к файлу, 0 - номер варпа
        try:
            tablet = simple(df_n[x][0],rout[n],w,warp)             # запуска фукнции по кругу
            vision = pd.concat([one,tablet])                # коннетация
            one = vision.copy()                           # копирование  (кеширование не работает без копирования)
            return tablet,vision,one
        except ValueError:
            next
        except AttributeError:
            next
        except TypeError:
            print('в базе данных кончились машины - алгоритм отработал все объекты счисления') 

    def car_number(df):
        count = df.Unic_gar.nunique(dropna=True)  # подсчёт уникальных номеров и
        return count

    op_time = car_number(df_n[x][0]) # формирование длинны кеша
        
    for _ in range(op_time):
        table,vision,one = call(one,df_n,warp,rout)    # служебная
     

    def lazy_me(df):                    # пост обработка
        lazy = df.copy()
        lazy = lazy.reset_index()
        del lazy['index']
        return lazy

    vision = lazy_me(vision)   
    
    return vision

In [13]:
def coub_02():   
    
    """13.03.2020 работает"""
    
    """16.03.2020 после преобразования  в 
    
       df['_ГарНомер'] = df['_ГарНомер'].astype('float')
       df['_ГарНомер'] = df['_ГарНомер'].astype('int64') 
            
       всё заработало кроме нвдс"""
    
    utt = coub_01(0,0,0)    
    nst = coub_01(1,1,1)    
    trans = coub_01(2,2,2)    
    nvds = coub_01(3,3,3)
    cher = coub_01(4,4,4)
    mtc = coub_01(5,5,5)
    mtk = coub_01(6,6,6)    
    sam = coub_01(7,7,7) 
    
    # 25.03.2020 добавлено промежуточное хранилище под vision
    
    utt.to_csv('C:/Users/Battlestation/ural/vision_utt.csv', 
                  encoding='utf-8', index=False)
    nst.to_csv('C:/Users/Battlestation/ural/vision_nst.csv', 
                  encoding='utf-8', index=False)
    trans.to_csv('C:/Users/Battlestation/ural/vision_trans.csv', 
                  encoding='utf-8', index=False)
    nvds.to_csv('C:/Users/Battlestation/ural/vision_nvds.csv', 
                  encoding='utf-8', index=False)
    cher.to_csv('C:/Users/Battlestation/ural/vision_cher.csv', 
                  encoding='utf-8', index=False)
    mtc.to_csv('C:/Users/Battlestation/ural/vision_mtc.csv', 
                  encoding='utf-8', index=False)
    mtk.to_csv('C:/Users/Battlestation/ural/vision_mtk.csv', 
                  encoding='utf-8', index=False)
    sam.to_csv('C:/Users/Battlestation/ural/vision_sam.csv', 
                  encoding='utf-8', index=False)
    
    return (utt,nst,trans,nvds,cher,mtc,mtk,sam)

In [14]:
# 19.03.2020 продолжаем под drink the sunny, работает от 20.03.2020

def wildfire(df,name):
    
    # 19.03.2020 внёс правки дома, без них не считало
    
    df['Номера ремонтных листов с периодичностью в 0 дней'] = [str(item) for item in df['Номера ремонтных листов с периодичностью в 0 дней']]
    df['Описание серьёзных проблем'] = [str(item) for item in df['Описание серьёзных проблем']]
    df['Описание проблем'] = [str(item) for item in df['Описание проблем']]
    df['Описание проблем в устойчивых циклах'] = [str(item) for item in df['Описание проблем в устойчивых циклах']]    

    # сохраняем витрину на новые машиы
    new_df = df[df['Это новая машина, vin'].notnull()]

    # количество новых машин прошедших ремонт
    new_car = df['Это новая машина, vin'].value_counts().sum()    

    # используем гаражный номер как уникальный индентификатор
    cars = df['Гаражный номер'].value_counts().sum()

    # простой по всем ремонтным операциям (по всем ремонтным листам) на одну машину
    cars_hold = df['Простой по всем ремонтным событиям'].sum()

    # количество открытых и закрытых ремонтных листов на одну машину
    rep_list_n = df['Кол-во постановок на ремонт'].sum()

    # количество выполненных ремонтных операций по всем ремонтным листам на одну машину
    rep_fix_n = df['Кол-во выполненых ремонтных операций'].sum()

    # количество уникальных типов ремонтных операциф выполненных на одну машину по всем ремонтным листам
    rep_un_n = df['Кол-во уникальных типов ремонтных операций'].sum()

    # количесвто уникальных типов ремонтных операций на эпизод - на одну постановку не ремонт на один отк.-зак. рем.лист
    rep_float = df['Доля уникальных ремонтных операций'].median()

    # количество повторяемых ремонтных операций,[циклов] - общее число типов ремонтных операций проведенных более 1 раза.
    rep_non_n = df['Кол-во циклов'].sum()  

    # общее число уникальных типов операций, которые повторяются в новых циклах
    new_cycl = df['Операции в новых циклах'].sum()

    # общее число уникальных типов операций, которые повторяються в устойчивых циклах
    stady_cycl = df['Операции в устойчивых циклах'].sum()

    # контрольная метрика - отношение суммы ремонтов чаще 10 дней к общему числу ремонтов реже 10 дней.
    neg_math = df['Доля негативной периодичности операций'].mean()

    # предел Ищенко, сумма случаев поставноки на ремонт чаще чем раз в 10 дней
    lower_10_d = df['Количество постановок на ремонт чаще 10 дней'].sum()

    # сумма случаев постановки на ремонт реже чем раз в 10 дней
    upper_10_d = df['Количество постановок на ремонт с нормальном интервалом'].sum()

    # расчётная граница допустимой периодичности - mediana аналога предела Ищенко, однако для каждой машины она своя
    math_th = df['Вероятно значение математического предела'].median()    

    # математика - доля нормальных ремонтов от общего числа чем больше - тем лучше
    math_good_status = df['Доля позитивной математической периодичности'].mean()

    # сумма всех постановок на ремонт с периодичностью ниже расчётной математической границы
    math_bad = df['Эпизоды ниже этой границы'].sum()

    # сумма всех постановок на ремонт с периодичностью выже расчётной метаматической границы
    math_good = df['Эпизоды выше этой границы'].sum()

    # ремонты, которые были завершены за 0 дней
    zero_d = df['Номера ремонтных листов с периодичностью в 0 дней'].value_counts().sum()   

    # цикличный ремонт двигателя ( в дработке)
    engine = df['Внимание! Цикличный ремонт двигателя'].sum() 

    # общее количество машин которые привыкли чинить чаще предела Ищенко
    alarm = df['AlaRm: ТС привыкли чинить чаще 10 дней'].sum()

    # цикличные машины, warning_1 между 20 и 5, warning_2 ниже 5,, warning_3 доля циклов больше 20%

    warning_1 =len(df[df['Оценка цикличности ТС'].str.contains('Внимание! Обратите внимание на циклы операций',na=False)])
    warning_2 =len(df[df['Оценка цикличности ТС'].str.contains('Внимание! Доля циклов мала, считает плохо',na=False)])
    warning_3 =len(df[df['Оценка цикличности ТС'].str.contains('Внимание! Цикличная машина - готовьте запчасти',na=False)])

    # разбивка по простою

    warning_4 = len(df[df['Характер простоя'].str.contains('Машина простояла в ремонте больше месяца',na=False)])
    warning_5 = len(df[df['Характер простоя'].str.contains('Машина простояла в ремонте больше квартала',na=False)])
    warning_6 = len(df[df['Характер простоя'].str.contains('Машина простояла в ремонте больше полугодия',na=False)])
    warning_7 = len(df[df['Характер простоя'].str.contains('Машина простояла в ремонте почти год',na=False)])    

    # сумма строгих негативных оценок работы РММ по циклам операций 
    warning_8 = df['Кол-во серьёзных проблем в РММ'].sum()    

    # сумма операций по этим оценкам
    base_error = df['Описание серьёзных проблем'].value_counts().sum()      

    # количество не строгикх плохихи оценок процесса
    mistake = df['Кол-во проблем в РММ'].sum()

    # описание не строгих проблем в РММ
    cycle_error = df['Описание проблем'].value_counts().sum()

    # сумма критических проблем - нурашения в устойчивых циклах
    crit_mistake = df['Количество проблем в устойчивых циклах'].sum() 

    # сумма операций в устойчивых циклах
    cycle_error_old = df['Описание проблем в устойчивых циклах'].value_counts().sum()  

    wildfire = pd.DataFrame([['Число отремонтированных ТС', cars], 
                                 ['Количество новых ТС побывавших в ремонте', new_car],
                                 ['Общий простой этих ТС в РММ', cars_hold],
                                 ['Кол-во постановок на ремонт', rep_list_n],
                                 ['Кол-во выполненых ремонтных операций', rep_fix_n],
                                 ['Кол-во уникальных типов ремонтных операций', rep_un_n],
                                 ['Кол-во повторяемых уникальных типов ремонтных операций', rep_non_n],

                                 ['Кол-во уникальных типов ремонтных операций на эпизод',rep_float],

                                 ['Число операции в устойчивых циклах', stady_cycl],
                                 ['Число операции в новых циклах', new_cycl],
                                 ['Доля негативной периодичности операций', neg_math],
                                 ['Количество постановок на ремонт чаще 10 дней', lower_10_d],
                                 ['Количество постановок на ремонт с нормальном интервалом', upper_10_d],                             

                                 ['Допустимоый период периодичности по версии машины',math_th],

                                 ['Доля позитивной математической периодичности', math_good_status],

                                 ['Число эпизоды ниже теоретической математической границы', math_bad],
                                 ['Число эпизодов выше теоретической математической границы', math_good],

                                 ['Количество постановок на ремонт за 0 дней', zero_d],
                                 ['Случаи цикличного ремонта двигателя', engine],
                                 ['ТС которые чинят чаще предела Ищенко', alarm],

                                 ['Общее сумма машин с высокой циклинчостью', warning_3],
                                 ['Общая мумма машин со средней цикличностью', warning_1],
                                 ['общая сумма машин с низкой цикличностью', warning_2],

                                 ['Число машин простоявщих больше месяца', warning_4],
                                 ['Число машин простоявщих больше квартала', warning_5],
                                 ['Число машин простоявщих больше полугодия', warning_6],
                                 ['Число машин простоявщих больше 9 ти месяцев', warning_7],

                                 ['Строгая негативная оценка РММ',warning_8],                             
                                 ['Количество операций по строгой оценке', base_error],

                                 ['Нагетивная оценка РММ', mistake],
                                 ['Количество опеhfций по негативной оценке',cycle_error],

                                 ['Количество нарушений в устойчивых циклах', crit_mistake],
                                 ['Количество операций по этим нарушениям', cycle_error_old],
                                 ],
                                 columns=['Метрики',name])

    pd.set_option('display.max_rows', len(wildfire))
    
    return wildfire

In [15]:
def soundfire(vision):
    
    # работает от 20.03.2020
    
    """
    откликаеться на info = soundfire(), возвращает только таблицы с числами! не путать с wildfire! который vision[0-7]
    
    сводка по обществам, с записью html файлов 
    
    info[0] - итоговая
    info[1] - utt
    info[2] - nst
    info[3] - trans
    info[4] - nvds
    info[5] - cher
    info[6] - mtc
    info[7] - mtk
    info[8] - sam
    """
    
    result_utt = wildfire(vision[0],"УТТ")
    result_nst = wildfire(vision[1],"НСТ") 
    result_trans = wildfire(vision[2],"ТРАНС")    
    result_nvds = wildfire(vision[3],"НВДС")    
    result_cher = wildfire(vision[4],"ЧЕР") 
    result_mtc = wildfire(vision[5],"МТС") 
    result_mtk = wildfire(vision[6],"МТК") 
    result_sam = wildfire(vision[7],"САМ")
    
    result_all = result_sam.merge(result_nst)
    result_all_1 = result_all.merge(result_trans)
    result_all_2 = result_all_1.merge(result_nvds)
    result_all_3 = result_all_2.merge(result_cher)
    result_all_4 = result_all_3.merge(result_mtc)
    result_all_5 = result_all_4.merge(result_mtk)
    result_all_6 = result_all_5.merge(result_utt)
    
    # путь на работе 'C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_test/общая_сводка.html'
    
    result_all_6.to_html('C:/Users/Battlestation/ural/общая_сводка.html')    
    result_utt.to_html('C:/Users/Battlestation/ural/сводка_утт.html')    
    result_nst.to_html('C:/Users/Battlestation/ural/сводка_нст.html')    
    result_trans.to_html('C:/Users/Battlestation/ural/сводка_транссервис.html')    
    result_nvds.to_html('C:/Users/Battlestation/ural/сводка_нвдс.html')    
    result_cher.to_html('C:/Users/Battlestation/ural/сводка_черногор.html')    
    result_mtc.to_html('C:/Users/Battlestation/ural/сводка_мтс.html')    
    result_mtk.to_html('C:/Users/Battlestation/ural/сводка_мтк.html')    
    result_sam.to_html('C:/Users/Battlestation/ural/сводка_самотор.html')
    
    return (result_all_6,result_utt,result_nst,result_trans, result_nvds, result_cher,result_mtc,result_mtk,result_sam)

In [16]:
 # цепляем фукнции профиля безопасности, почти работает - ругаеться на NameError в самом низу 20.03.2020
    
def recon(info):
    recon = info.describe()   
    
    """recon_hard события которые строго отвечают всем условиям"""

    mask = ((info['Кол-во постановок на ремонт'] > recon['Кол-во постановок на ремонт'][6]) & 
                (info['Кол-во выполненых ремонтных операций'] > recon['Кол-во выполненых ремонтных операций'][6]) &
                (info['Количество постановок на ремонт чаще 10 дней'] > recon['Количество постановок на ремонт чаще 10 дней'][6])&
                (info['Простой по всем ремонтным событиям'] > pd.Timedelta(recon['Простой по всем ремонтным событиям'][6],unit='d')))
    recon_hard = info[mask]
        
    pd.set_option('display.max_colwidth', -1)        
    pd.set_option('display.max_rows', recon_hard.shape[0])

    """recon_soft события которые НЕ-строго отвечают всем условиям"""

    mask = ((info['Кол-во постановок на ремонт'] > recon['Кол-во постановок на ремонт'][6]) | 
                (info['Кол-во выполненых ремонтных операций'] > recon['Кол-во выполненых ремонтных операций'][6]) |
                (info['Количество постановок на ремонт чаще 10 дней'] > recon['Количество постановок на ремонт чаще 10 дней'][6])|
                (info['Простой по всем ремонтным событиям'] > pd.Timedelta(recon['Простой по всем ремонтным событиям'][6],unit='d')))
    recon_soft = info[mask]

    """собтия с множеством постановок в 0 дней """

    mask = ((info['Количество постановок на ремонт с интервалом 0 дней'] >= recon['Количество постановок на ремонт с интервалом 0 дней'][7]))
    recon_zero = info[mask] 

    # при подсчёте в лоб ругаеться на ограничение целочисленной математики,нужен модуль decimal и мне лень эт описать

    """
    hard_val_percent = 100 * recon_hard['Простой по всем ремонтным событиям'].sum() / info['Простой по всем ремонтным событиям'].sum()
    soft_val_percent = 100 * recon_soft['Простой по всем ремонтным событиям'].sum() / info['Простой по всем ремонтным событиям'].sum() 

    """      

    all_time = info['Простой по всем ремонтным событиям'].sum()

    good_time = all_time - recon_soft['Простой по всем ремонтным событиям'].sum()

    columns = ['Доля уникальных ремонтных операций', 'Кол-во циклов','Число устойчивых циклов',
                   'Число потенциальных циклов', 'Операции в устойчивых циклах','Операции в новых циклах',
                   'Вероятно значение математического предела','Эпизоды ниже этой границы', 'Эпизоды выше этой границы',
                   'Номера ремонтных листов ниже математического предела',
                   'Доля позитивной математической периодичности',    
                   'Оценка ценности IQL',
                   'Это новая машина, vin', 'год_выпуска',  
                   'поставщик', 'Внимание! Цикличный ремонт двигателя',
                    'AlaRm: ТС привыкли чинить чаще 10 дней',
                   'Кол-во серьёзных проблем в РММ',
                   'Описание серьёзных проблем',
                   'Кол-во проблем в РММ',
                   'Описание проблем',  
                   'Оценка цикличности ТС', 'Количество проблем в устойчивых циклах',
                   'Описание проблем в устойчивых циклах',
                   'Доля негативной периодичности операций',
                   'Характер простоя']       
        
    columns_2 = ['Доля уникальных ремонтных операций', 'Кол-во циклов','Число устойчивых циклов',
                   'Число потенциальных циклов', 'Операции в устойчивых циклах','Операции в новых циклах',
                   'Вероятно значение математического предела','Эпизоды ниже этой границы', 'Эпизоды выше этой границы',
                   'Номера ремонтных листов ниже математического предела',
                   'Доля позитивной математической периодичности',    
                   'Оценка ценности IQL', 'Внимание! Цикличный ремонт двигателя',
                   'AlaRm: ТС привыкли чинить чаще 10 дней',
                   'Кол-во серьёзных проблем в РММ',
                   'Описание серьёзных проблем',
                   'Кол-во проблем в РММ',
                   'Описание проблем',                

                   'Оценка цикличности ТС', 'Количество проблем в устойчивых циклах',
                   'Описание проблем в устойчивых циклах',
                   'Доля негативной периодичности операций',
                   'Характер простоя']

    recon_hard.drop(columns, inplace=True, axis=1)
    recon_soft.drop(columns, inplace=True, axis=1)
    recon_zero.drop(columns, inplace=True, axis=1)

    recon_hard = recon_hard.sort_values('Простой по всем ремонтным событиям',ascending=False)
    recon_soft = recon_soft.sort_values('Простой по всем ремонтным событиям',ascending=False)
    recon_zero = recon_zero.sort_values('Простой по всем ремонтным событиям',ascending=False)       
        
    print("Сумма простоя по жесткому фильтру: " , recon_hard['Простой по всем ремонтным событиям'].sum(), '\n',
              "Сумма простоя по мягкому фильтру: ", recon_soft['Простой по всем ремонтным событиям'].sum(), '\n',
              "Предпологаемый чистый простой: ", good_time, sep="")  
        
    # пробуем засунуть новые машины в эту функцию
    
    # nvds_1 = фильтр по машинам с постановкой чаще чем раз в 10 дней
    
    # nvds_2 = новые машины
    
    nvds_1 = info[info['Количество постановок на ремонт чаще 10 дней'] >0]

    nvds_2 = info[info['Это новая машина, vin'].notnull()]

    mask = ((info['Количество постановок на ремонт чаще 10 дней']>0) & 
                (info['Это новая машина, vin'].notnull()))          
    recon_nvds = info[mask]
        
    nvds_1.drop(columns, inplace=True, axis=1)
    nvds_2.drop(columns, inplace=True, axis=1)
    recon_nvds.drop(columns_2, inplace=True, axis=1)

    nvds_1 = nvds_1.sort_values('Простой по всем ремонтным событиям',ascending=False)
    nvds_2 = nvds_2.sort_values('Простой по всем ремонтным событиям',ascending=False)
        
        # сбоило здесь, внёс правки ниже чтения файла в оснвоном алгоритме   
        
    mask = ((info['Количество постановок на ремонт чаще 10 дней']>0) & 
                (info['Это новая машина, vin'].notnull()))          
    recon_nvds_1 = info[mask]

    mask = ((recon_nvds_1['Кол-во постановок на ремонт'] > recon['Кол-во постановок на ремонт'][6]) | 
                (recon_nvds_1['Кол-во выполненых ремонтных операций'] > recon['Кол-во выполненых ремонтных операций'][6]) |
                (recon_nvds_1['Количество постановок на ремонт чаще 10 дней'] > recon['Количество постановок на ремонт чаще 10 дней'][6])|
                (recon_nvds_1['Простой по всем ремонтным событиям'] > pd.Timedelta(recon['Простой по всем ремонтным событиям'][6],unit='d')))
    recon_native = recon_nvds_1[mask]
        
    recon_native.drop(columns_2, inplace=True, axis=1)
        
    # здесь была сортировка, мне пришлось её выкинуть потому что с ней алгоритм засбоил после правок 2020 года
                
    pd.set_option('display.max_colwidth', -1)  
    pd.set_option('display.max_rows', None) 
        
    # блок закупки
        
    recon_test = info
        
    recon_test = info[info['Это новая машина, vin'].notnull()]
        
    columns_3 = ['Доля уникальных ремонтных операций', 'Кол-во циклов','Число устойчивых циклов',
                   'Число потенциальных циклов', 'Операции в устойчивых циклах','Операции в новых циклах',
                   'Вероятно значение математического предела','Эпизоды ниже этой границы', 'Эпизоды выше этой границы',
                   'Номера ремонтных листов ниже математического предела',
                   'Доля позитивной математической периодичности',    
                   'Оценка ценности IQL',
                   'Внимание! Цикличный ремонт двигателя',
                   'Кол-во серьёзных проблем в РММ',
                   'Описание серьёзных проблем',
                   'Кол-во проблем в РММ',
                   'Описание проблем',  
                   'Оценка цикличности ТС', 'Количество проблем в устойчивых циклах',
                   'Описание проблем в устойчивых циклах',
                   'Доля негативной периодичности операций',
                   'Характер простоя'] 
        
    recon_test.поставщик = recon_test.поставщик.astype('str')
        
    recon_test.drop(columns_3, inplace=True, axis=1)
        
    # vsego == все купленная техкника которая ходила в ремонт
    
    vsego = len(recon_test)
        
    # вынимаем новые машины из всех.
        
    recon_test['год_выпуска'] = recon_test['год_выпуска'].apply(lambda x: pd.to_datetime(x[0],errors = 'coerce'))
        
    start = datetime.datetime( 2017,1, 1)
    end = datetime.datetime(2020 , 12, 31)
    mask = (recon_test['год_выпуска'] > start) & (recon_test['год_выпуска'] <= end)
    recon_test = recon_test.loc[mask] 
        
    # recon_test == 3х летки побывавшие в ремонте
        
    bad_info = vsego - len(recon_test)
        
    print('Техника старше 3х лет: ', bad_info,'\n','\n',)
        
    return (recon_hard, recon_soft, recon_zero, nvds_1, nvds_2, recon_nvds, recon_native,recon_test)  
    
    # вызываем функции в отношении всех объектов счисления
    
def anime_coub(vision,info):
    
    # добавлено чтение result_loss - глупо таскать его по кешу, коль скоро он уже есть
    
    result_loss = pd.read_csv('C:/Users/Battlestation/ural/result_1c.csv', sep=',', encoding="utf-8") 
    
    result_loss.rename(columns={result_loss.columns[0]:'Поля рем.листов'},inplace=True)
    
    recon_utt = recon(vision[0])
    recon_nst = recon(vision[1])
    recon_trans = recon(vision[2])
    recon_nvds = recon(vision[3])
    recon_cher = recon(vision[4])
    recon_mtc = recon(vision[5])
    recon_mtk = recon(vision[6])
    recon_sam = recon(vision[7])  
    
    # мой путь на работе C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_test/
    
    # recon_hard
    
    recon_utt[0].to_html('C:/Users/Battlestation/ural/строгий_фильтр_утт.html')    
    recon_nst[0].to_html('C:/Users/Battlestation/ural/строгий_фильтр_нст.html')    
    recon_trans[0].to_html('C:/Users/Battlestation/ural/строгий_фильтр_транс.html')    
    recon_nvds[0].to_html('C:/Users/Battlestation/ural/строгий_фильтр_нвдс.html')    
    recon_cher[0].to_html('C:/Users/Battlestation/ural/строгий_фильтр_чер.html')    
    recon_mtc[0].to_html('C:/Users/Battlestation/ural/строгий_фильтр_мтс.html')    
    recon_mtk[0].to_html('C:/Users/Battlestation/ural/строгий_фильтр_мтк.html')    
    recon_sam[0].to_html('C:/Users/Battlestation/ural/строгий_фильтр_сам.html')    
    
    # recon_soft
    
    recon_utt[1].to_html('C:/Users/Battlestation/ural/мягкий_фильтр_утт.html')    
    recon_nst[1].to_html('C:/Users/Battlestation/ural/мягкий_фильтр_нст.html')    
    recon_trans[1].to_html('C:/Users/Battlestation/ural/мягкий_фильтр_транс.html')    
    recon_nvds[1].to_html('C:/Users/Battlestation/ural/мягкий_фильтр_нвдс.html')    
    recon_cher[1].to_html('C:/Users/Battlestation/ural/мягкий_фильтр_чер.html')    
    recon_mtc[1].to_html('C:/Users/Battlestation/ural/мягкий_фильтр_мтс.html')    
    recon_mtk[1].to_html('C:/Users/Battlestation/ural/мягкий_фильтр_мтк.html')    
    recon_sam[1].to_html('C:/Users/Battlestation/ural/мягкий_фильтр_сам.html')    
    
    #recon_zero
    
    recon_utt[2].to_html('C:/Users/Battlestation/ural/нули_утт_сводка.html')    
    recon_nst[2].to_html('C:/Users/Battlestation/ural/нули_нст_сводка.html')    
    recon_trans[2].to_html('C:/Users/Battlestation/ural/нули_транс_сводка.html')    
    recon_nvds[2].to_html('C:/Users/Battlestation/ural/нули_нвдс_сводка.html')    
    recon_cher[2].to_html('C:/Users/Battlestation/ural/нули_чер_сводка.html')    
    recon_mtc[2].to_html('C:/Users/Battlestation/ural/нули_мтс_сводка.html')    
    recon_mtk[2].to_html('C:/Users/Battlestation/ural/нули_мтк_сводка.html')    
    recon_sam[2].to_html('C:/Users/Battlestation/ural/нули_сам_сводка.html')   
    
    # nvds_1
        
    recon_utt[3].to_html('C:/Users/Battlestation/ural/частые_ремонты_утт.html')    
    recon_nst[3].to_html('C:/Users/Battlestation/ural/частые_ремонты_нст.html')    
    recon_trans[3].to_html('C:/Users/Battlestation/ural/частые_ремонты_транс.html')    
    recon_nvds[3].to_html('C:/Users/Battlestation/ural/частые_ремонты_нвдс.html')    
    recon_cher[3].to_html('C:/Users/Battlestation/ural/частые_ремонты_чер.html')    
    recon_mtc[3].to_html('C:/Users/Battlestation/ural/частчые_ремонты_мтс.html')    
    recon_mtk[3].to_html('C:/Users/Battlestation/ural/частые_ремонты_мтк.html')    
    recon_sam[3].to_html('C:/Users/Battlestation/ural/частые_ремонты_сам.html')   
    
    # nvds_2
    
    recon_utt[4].to_html('C:/Users/Battlestation/ural/новые_машины_утт.html')    
    recon_nst[4].to_html('C:/Users/Battlestation/ural/новые_машины_нст.html')    
    recon_trans[4].to_html('C:/Users/Battlestation/ural/новые_машины_транс.html')    
    recon_nvds[4].to_html('C:/Users/Battlestation/ural/новые_машины_нвдс.html')    
    recon_cher[4].to_html('C:/Users/Battlestation/ural/новые_машины_чер.html')    
    recon_mtc[4].to_html('C:/Users/Battlestation/ural/новые_машины_мтс.html')    
    recon_mtk[4].to_html('C:/Users/Battlestation/ural/новые_машины_мтк.html')    
    recon_sam[4].to_html('C:/Users/Battlestation/ural/новые_машины_сам.html')
  
    # recon nvds
    
    recon_utt[5].to_html('C:/Users/Battlestation/ural/частые_ремонт_новые_машины_утт.html')    
    recon_nst[5].to_html('C:/Users/Battlestation/ural/частые_ремонт_новые_машины_нст.html')    
    recon_trans[5].to_html('C:/Users/Battlestation/ural/частые_ремонт_новые_машины_транс.html')    
    recon_nvds[5].to_html('C:/Users/Battlestation/ural/частые_ремонт_новые_машины_нвдс.html')    
    recon_cher[5].to_html('C:/Users/Battlestation/ural/частые_ремонт_новые_машиный_чер.html')    
    recon_mtc[5].to_html('C:/Users/Battlestation/ural/частые_ремонт_новые_машины_мтс.html')    
    recon_mtk[5].to_html('C:/Users/Battlestation/ural/частые_ремонт_новые_машины_мтк.html')    
    recon_sam[5].to_html('C:/Users/Battlestation/ural/частые_ремонт_новые_машины_сам.html')
   
    # recon_native
    
    recon_utt[6].to_html('C:/Users/Battlestation/ural/новые_машины_мягкий_фильтр_утт.html')    
    recon_nst[6].to_html('C:/Users/Battlestation/ural/новые_машины__мягкий_фильтр_нст.html')    
    recon_trans[6].to_html('C:/Users/Battlestation/ural/новые_машины_мягкий_фильтр_транс.html')    
    recon_nvds[6].to_html('C:/Users/Battlestation/ural/новые_машины_мягкий_фильтр_нвдс.html')    
    recon_cher[6].to_html('C:/Users/Battlestation/ural/новые_машины_мягкий_фильтр_чер.html')    
    recon_mtc[6].to_html('C:/Users/Battlestation/ural/новые_машины_мягкий_фильтр_мтс.html')    
    recon_mtk[6].to_html('C:/Users/Battlestation/ural/новые_машины_мягкий_фильтр_мтк.html')    
    recon_sam[6].to_html('C:/Users/Battlestation/ural/новые_машины_мягкий_фильтр_сам.html') 
    
    # test
    
    recon_utt[7].to_html('C:/Users/Battlestation/ural/3хлетки_сводка_утт.html')    
    recon_nst[7].to_html('C:/Users/Battlestation/ural/3хлетки_сводка_нст.html')    
    recon_trans[7].to_html('C:/Users/Battlestation/ural/3хлетки_сводка_транс.html')    
    recon_nvds[7].to_html('C:/Users/Battlestation/ural/3хлетки_сводка_нвдс.html')    
    recon_cher[7].to_html('C:/Users/Battlestation/ural/3хлетки_сводка_чер.html')    
    recon_mtc[7].to_html('C:/Users/Battlestation/ural/3хлетки_сводка_мтс.html')    
    recon_mtk[7].to_html('C:/Users/Battlestation/ural/3хлетки_сводка_мтк.html')    
    recon_sam[7].to_html('C:/Users/Battlestation/ural/3хлетки_сводка_сам.html')     
    
    # создаём витрину оценки положения экономической безопасности.
        
    self_defense= pd.DataFrame([['Коррупция',
                             recon_sam[0]['Простой по всем ремонтным событиям'].sum(),len(recon_sam[0]),
                             recon_nst[0]['Простой по всем ремонтным событиям'].sum(),len(recon_nst[0]),
                             recon_trans[0]['Простой по всем ремонтным событиям'].sum(),len(recon_trans[0]),
                             recon_nvds[0]['Простой по всем ремонтным событиям'].sum(),len(recon_nvds[0]),
                             recon_cher[0]['Простой по всем ремонтным событиям'].sum(),len(recon_cher[0]),
                             recon_mtc[0]['Простой по всем ремонтным событиям'].sum(),len(recon_mtc[0]),
                             recon_mtk[0]['Простой по всем ремонтным событиям'].sum(),len(recon_mtk[0]),
                             recon_utt[0]['Простой по всем ремонтным событиям'].sum(),len(recon_utt[0]),],
                            
                            
                            ['Махинации', 
                             recon_sam[1]['Простой по всем ремонтным событиям'].sum(),len(recon_sam[1]),
                             recon_nst[1]['Простой по всем ремонтным событиям'].sum(),len(recon_nst[1]),
                             recon_trans[1]['Простой по всем ремонтным событиям'].sum(),len(recon_trans[1]),
                             recon_nvds[1]['Простой по всем ремонтным событиям'].sum(),len(recon_nvds[1]),
                             recon_cher[1]['Простой по всем ремонтным событиям'].sum(),len(recon_cher[1]),
                             recon_mtc[1]['Простой по всем ремонтным событиям'].sum(),len(recon_mtc[1]),
                             recon_mtk[1]['Простой по всем ремонтным событиям'].sum(),len(recon_mtk[1]),
                             recon_utt[1]['Простой по всем ремонтным событиям'].sum(),len(recon_utt[1]),],
                            
                            
                            ['Закупки техники',
                             recon_sam[6]['Простой по всем ремонтным событиям'].sum(),len(recon_sam[6]),
                             recon_nst[6]['Простой по всем ремонтным событиям'].sum(),len(recon_nst[6]),
                             recon_trans[6]['Простой по всем ремонтным событиям'].sum(),len(recon_trans[6]),
                             recon_nvds[6]['Простой по всем ремонтным событиям'].sum(),len(recon_nvds[6]),
                             recon_cher[6]['Простой по всем ремонтным событиям'].sum(), len(recon_cher[6]),
                             recon_mtc[6]['Простой по всем ремонтным событиям'].sum(),len(recon_mtc[6]),
                             recon_mtk[6]['Простой по всем ремонтным событиям'].sum(),len(recon_mtk[6]),
                             recon_utt[6]['Простой по всем ремонтным событиям'].sum(),len(recon_utt[6]),],], 
                               
                            columns=['Метрики безопасности', 
                                     "САМ Простой",'САМ Кол.ТС', "НСТ Простой",'НСТ Кол.ТС',
                                     "ТРА Простой",'ТРА Кол.ТС', "НВДС Простой",'НВДС Кол.ТС',
                                     "ЧЕР Простой", 'ЧЕР Кол.ТС',"МТС Простой", 'МТС Кол.ТС', 
                                     "МТК Простой",'МТК Кол.ТС', "УТТ Простой",'УТТ Кол.ТС'])
    
    # 20.03.2020 добавлено result_all_6 = info[0]  
    
    # соблюдайте регистр и раскладку, весь найминг в колонках написан на кириллице, при имён смене с 313 по 402 ругаеться
    
    result_all_6 = info[0] 
    
    self_control=pd.DataFrame([['Пооблемы ',
                           float('{:.3f}'.format(result_loss['% of Total Values САМ'].sum())),
                           float('{:.3f}'.format(100 * self_defense['САМ Простой'][0].days /  result_all_6.САМ[2].days)),
                                                 
                           float('{:.3f}'.format(result_loss['% of Total Values НСТ'].sum())),
                           float('{:.3f}'.format(100 * self_defense['НСТ Простой'][0].days /  result_all_6.НСТ[2].days)),
                        
                           float('{:.3f}'.format(result_loss['% of Total Values ТРА'].sum())),
                           float('{:.3f}'.format(100 * self_defense['ТРА Простой'][0].days /  result_all_6.ТРАНС[2].days)),
                                                
                           float('{:.3f}'.format(result_loss['% of Total Values НВДС'].sum())),
                           float('{:.3f}'.format(100 * self_defense['НВДС Простой'][0].days /  result_all_6.НВДС[2].days)),
                                                 
                           float('{:.3f}'.format(result_loss['% of Total Values ЧЕР'].sum())),
                           float('{:.3f}'.format(100 * self_defense['ЧЕР Простой'][0].days /  result_all_6.ЧЕР[2].days)),
                                                 
                           float('{:.3f}'.format(result_loss['% of Total Values МТС'].sum())),
                           float('{:.3f}'.format(100 * self_defense['МТС Простой'][0].days /  result_all_6.МТС[2].days)),
                                                 
                           float('{:.3f}'.format(result_loss['% of Total Values МТК'].sum())),
                           float('{:.3f}'.format(100 * self_defense['МТК Простой'][0].days /  result_all_6.МТК[2].days)),
                                                 
                           float('{:.3f}'.format(result_loss['% of Total Values УТТ'].sum())),
                           float('{:.3f}'.format(100 * self_defense['УТТ Простой'][0].days /  result_all_6.УТТ[2].days)),],],
                                                       
                              
                           columns=['Сводка безопасности', 
                                     "САМ Учёт",'САМ Простой', "NST Учёт",'NST Простой',
                                     "Транс Учёт",'ТРА Простой', "НВДС Учёт",'НВДС Простой',
                                     "ЧАТ Учёт", 'ЧАТ Простой',"MTC Учёт", 'MTC Простой', 
                                     "MTK Учёт",'МТК Простой', "УТТ Учёт",'УТТ Простой'])
    
    anime = pd.DataFrame({'ДО' :['САМ',"НСТ","ТРА","НВДС","ЧЕР","МТС","МТК","УТТ"],
                        "НАРУШЕНИЕ УЧЁТА %":[float('{:.3f}'.format(result_loss['% of Total Values САМ'].sum())),
                               float('{:.3f}'.format(result_loss['% of Total Values НСТ'].sum())),
                               float('{:.3f}'.format(result_loss['% of Total Values ТРА'].sum())),
                               float('{:.3f}'.format(result_loss['% of Total Values НВДС'].sum())),
                               float('{:.3f}'.format(result_loss['% of Total Values ЧЕР'].sum())),
                               float('{:.3f}'.format(result_loss['% of Total Values МТС'].sum())),
                               float('{:.3f}'.format(result_loss['% of Total Values МТК'].sum())),
                               float('{:.3f}'.format(result_loss['% of Total Values УТТ'].sum()))],
                         
                        "ПРОСТОЙ КОРРУПЦИЯ %":[float('{:.3f}'.format(100 * self_defense['САМ Простой'][0].days /  result_all_6.САМ[2].days)),
                                      float('{:.3f}'.format(100 * self_defense['НСТ Простой'][0].days /  result_all_6.НСТ[2].days)),
                                      float('{:.3f}'.format(100 * self_defense['ТРА Простой'][0].days /  result_all_6.ТРАНС[2].days)),
                                      float('{:.3f}'.format(100 * self_defense['НВДС Простой'][0].days /  result_all_6.НВДС[2].days)),
                                      float('{:.3f}'.format(100 * self_defense['ЧЕР Простой'][0].days /  result_all_6.ЧЕР[2].days)),
                                      float('{:.3f}'.format(100 * self_defense['МТС Простой'][0].days /  result_all_6.МТС[2].days)),
                                      float('{:.3f}'.format(100 * self_defense['МТК Простой'][0].days /  result_all_6.МТК[2].days)),
                                      float('{:.3f}'.format(100 * self_defense['УТТ Простой'][0].days /  result_all_6.УТТ[2].days))],
                         
                        "ПРОСТОЙ МАХИНАЦИИ %":[float('{:.3f}'.format(100 * self_defense['САМ Простой'][1].days /  result_all_6.САМ[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['НСТ Простой'][1].days /  result_all_6.НСТ[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['ТРА Простой'][1].days /  result_all_6.ТРАНС[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['НВДС Простой'][1].days /  result_all_6.НВДС[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['ЧЕР Простой'][1].days /  result_all_6.ЧЕР[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['МТС Простой'][1].days /  result_all_6.МТС[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['МТК Простой'][1].days /  result_all_6.МТК[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['УТТ Простой'][1].days /  result_all_6.УТТ[2].days))],
                         
                        "ППРОСТОЙ НОВЫЕ ТС %":[float('{:.3f}'.format(100 * self_defense['САМ Простой'][2].days /  result_all_6.САМ[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['НСТ Простой'][2].days /  result_all_6.НСТ[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['ТРА Простой'][2].days /  result_all_6.ТРАНС[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['НВДС Простой'][2].days /  result_all_6.НВДС[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['ЧЕР Простой'][2].days /  result_all_6.ЧЕР[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['МТС Простой'][2].days /  result_all_6.МТС[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['МТК Простой'][2].days /  result_all_6.МТК[2].days)),
                                     float('{:.3f}'.format(100 * self_defense['УТТ Простой'][2].days /  result_all_6.УТТ[2].days))],
                     
                     
                     "Число ТС по коррупции":[self_defense['САМ Кол.ТС'][0],
                             self_defense['НСТ Кол.ТС'][0],
                             self_defense['ТРА Кол.ТС'][0],
                             self_defense['НВДС Кол.ТС'][0],
                             self_defense['ЧЕР Кол.ТС'][0],
                             self_defense['МТС Кол.ТС'][0],
                               self_defense['МТК Кол.ТС'][0],
                             self_defense['УТТ Кол.ТС'][0],],
                      
                      "Число ТС по махинациям":[self_defense['САМ Кол.ТС'][1],self_defense['НСТ Кол.ТС'][1],self_defense['ТРА Кол.ТС'][1],
                               self_defense['НВДС Кол.ТС'][1],self_defense['ЧЕР Кол.ТС'][1],self_defense['МТС Кол.ТС'][1],
                               self_defense['МТК Кол.ТС'][1],self_defense['УТТ Кол.ТС'][1],],
                      
                      "Число закупленных ТС ":[self_defense['САМ Кол.ТС'][2],self_defense['НСТ Кол.ТС'][2],self_defense['ТРА Кол.ТС'][2],
                               self_defense['НВДС Кол.ТС'][2],self_defense['ЧЕР Кол.ТС'][2],self_defense['МТС Кол.ТС'][2],
                               self_defense['МТК Кол.ТС'][2],self_defense['УТТ Кол.ТС'][2],],})
    
    anime.index = anime.ДО
    anime = anime.drop(['ДО'], axis=1)
    
    result_loss.rename(columns={result_loss.columns[0]:'Поля рем.листов'},inplace=True)
    
    ad = result_loss.copy()
    
    ad['% of Total Values САМ'] = ad['% of Total Values САМ'].astype('str')
    ad['% of Total Values САМ'] = ad['% of Total Values САМ'].values +"%"

    ad['% of Total Values МТК'] = ad['% of Total Values МТК'].astype('str')
    ad['% of Total Values МТК'] = ad['% of Total Values МТК'].values +"%"

    ad['% of Total Values МТС'] = ad['% of Total Values МТС'].astype('str')
    ad['% of Total Values МТС'] = ad['% of Total Values МТС'].values +"%"

    ad['% of Total Values ЧЕР'] = ad['% of Total Values ЧЕР'].astype('str')
    ad['% of Total Values ЧЕР'] = ad['% of Total Values ЧЕР'].values +"%"    
    
    """блок вывода профиля безопасности"""
    
    recon_utt[7].to_html('C:/Users/Battlestation/ural/3хлетки_сводка_утт.html')    
    recon_nst[7].to_html('C:/Users/Battlestation/ural/3хлетки_сводка_нст.html')  
    
    # пользуйтесь self_defense[2].loc['САМ'] работе с обзорной сводкой безопасности    
    
    print("\n","\n","Отправить Ю.А Мякишева в ДО: " ,'\n','\n', anime.max())   
    
    return (recon_sam, recon_mtk, recon_mtc, recon_cher, recon_nvds, recon_trans, recon_nst,recon_utt),(result_loss, self_defense, anime, ad)

In [17]:
# нужна ещё одна фукнция!

# вокруг этого eva_04 = df.loc[df['_ГарНомер'].isin(car_in._gar.values)]

# и этого car_in.rename(columns={car_in.columns[0]:'_gar'},inplace=True)

# и этого car_in = car_in.drop_duplicates(['_gar'], keep='last')

def no_volkov_sql():
    
    """не буду им больше ничего объяснять, пускай за 5 минут на скуле делают    
       и комментить по человечески больше не буду, тото я смотрю 6 программистов пришлось нанять в филиал    
       без них за 5 минут на скуле не получаеться чет у моего начальника. https://youtu.be/nRoSiatdzlA?t=6
    """
    
    # мой путь на работе C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/2020_OS/NST_OS.xls
    
    car_in_sam = pd.read_excel('C:/Users/Battlestation/ural_2020/SAM_OS.xls', index_col=0,
                        dtype={'Name': str, 'Value': float})
    car_in_mtk = pd.read_excel('C:/Users/Battlestation/ural_2020/MTK_OS.xls', index_col=0,
                        dtype={'Name': str, 'Value': float})
    car_in_cha = pd.read_excel('C:/Users/Battlestation/ural_2020/CHA_OS.xls', index_col=0,
                        dtype={'Name': str, 'Value': float})
    car_in_mtc = pd.read_excel('C:/Users/Battlestation/ural_2020/MTC_OS.xls', index_col=0,
                        dtype={'Name': str, 'Value': float})
    car_in_nst = pd.read_excel('C:/Users/Battlestation/ural_2020/NST_OS.xls', index_col=0,
                        dtype={'Name': str, 'Value': float})
    car_in_nvds = pd.read_excel('C:/Users/Battlestation/ural_2020/NVDS_OS.xls', index_col=0,
                        dtype={'Name': str, 'Value': float})
    car_in_utt = pd.read_excel('C:/Users/Battlestation/ural_2020/UTT_OS.xls', index_col=0,
                        dtype={'Name': str, 'Value': float})
    
    car_in_trans = pd.read_excel('C:/Users/Battlestation/ural_2020/NST_OS.xls', index_col=0,
                        dtype={"Name": str,'value': float})
        
    def no_5_minute_sql(df):
        """фукнция возвращает все 3х летние машины"""
        
        df.rename(columns={df.columns[0]:'_gar'},inplace=True)
        df = df.drop_duplicates(['_gar'], keep='last')
        
        df['Дата выпуска (постройки)'] = pd.to_datetime(df['Дата выпуска (постройки)'], errors='coerce') 

        start = datetime.datetime( 2017,1, 1)
        end = datetime.datetime(2020 , 12, 31)
        mask = (df['Дата выпуска (постройки)'] > start) & (df['Дата выпуска (постройки)'] <= end)
        test = df.loc[mask]
        
        result = test
        
        return result
    
    car_in_sam = no_5_minute_sql(car_in_sam)
    car_in_mtk = no_5_minute_sql(car_in_mtk)
    car_in_cha = no_5_minute_sql(car_in_cha)
    car_in_mtc = no_5_minute_sql(car_in_mtc)
    car_in_nst = no_5_minute_sql(car_in_nst)
    car_in_nvds = no_5_minute_sql(car_in_nvds)
    car_in_utt = no_5_minute_sql(car_in_utt)
    car_in_trans = no_5_minute_sql(car_in_trans)
    
    """
    [0] - Саотлортранс
    [1] - МТК
    [2] - Черногор
    [3] - Мтс
    [4] - Нст
    [5] - Нвдс
    [6] - Утт
    [7] - Транс
    
    """
    
    return (car_in_sam,car_in_mtk,car_in_cha,car_in_mtc,car_in_nst,car_in_nvds,car_in_utt,car_in_trans)

def simple_empty_soul(all_df,fix_df):
    
    """
    [~]  Это логический оператор для NOT. 
    Таким образом, это создаст новый dataframe, когда значения df1["column_name"] не будут найдены в df2["column_name"]
    В результате этой логической операции мы получим витрину в которой будду лежать 3х летние машины не ходившие в РММ
    
    all_df  это test[n] из функции выше, содержит все 3х летние машины которые закупили для ДО
    fix_df  содержит все 3х летние машины которые ходили в ремонт, извлекается из основного алгоритма
    
    check_df результат, глдя на которой мы увидим технику, которая не ломаеться, но которая была закуплена ранее
    при этом необходимо учитыват ьчто исходные алгоритм rapid_fire не смог прочитать все данные по закупкам
    
    как следствие этого некоторые из витрин в выводе могут быть пустыми или ленивыми.    
    """
    
    new_df = all_df[~all_df["_gar"].isin(fix_df["Гаражный номер"])]
    result = len(new_df)
    controll_sum = len(fix_df)
    
    print("Проверка:",'\n','\n',        
        
          "Число техники, которая не сломалась будучи закупленной: ",result,'\n',
          "Число техники, которая ходила в ремонт будучи закупленной: ",controll_sum,'\n' )
    
    
    return new_df

In [18]:
# работает, если запускать последовательно 27 03 2020

def n9(vision):         
    
    
    """self_defense[0] - учётная
       self_defense[1] - простой по метрикам безопасности
       self_defense[2] - оценочная
       self_defense[3] - дуль учётной, необходим что бы порезать таблицу по обществам.
       
       info[0] - общая
       info[1] - утт
       info[2] - нст
       info[3] - транс
       info[4] - нвдс
       info[5] - чер
       info[6] - мтс
       info[7] - мтк
       info[8] - сам
       
       svodki[0][0] - сам
       svodki[1][0] - мтк
       svodki[2][0] - мтс
       svodki[3][0] - чер
       svodki[4][0] - нвдс
       svodki[5][0] - транс
       svodki[6][0] - нст
       svodki[7][0] - утт   
       
       svodki[0][0] - recon_hard 
       svodki[0][1] - recon_soft
       svodki[0][2] - recon_zero
       svodki[0][3] - nvds_1
       svodki[0][4] - nvds_2
       svodki[0][5] - recon_nvds
       svodki[0][6] - recon_native
       svodki[0][7] - recon_test
       
       check[0] - sam
       check[1] - mtk
       check[2] - che
       check[3] - mtc
       check[4] - nvds
       check[5] - trans
       check[6] - nst
       check[7] - utt       
       
       """
    
    info = soundfire(vision)
    svodki, self_defense = anime_coub(vision,info)
    test = no_volkov_sql()   
    
    # 23.03.2020 (внимание на нумерацию сводок)    
    
    
    check_sam = simple_empty_soul(test[0],   svodki[0][7])  
    check_mtk = simple_empty_soul(test[1], svodki[1][7])
    check_che = simple_empty_soul(test[2], svodki[3][7])
    check_mtc = simple_empty_soul(test[3], svodki[2][7]) # здесь всё правильно, не  сломанных может бытб больше чем сломаных
    check_nvds = simple_empty_soul(test[5], svodki[4][7])
    check_trans = simple_empty_soul(test[7],svodki[5][7])
    check_nst = simple_empty_soul(test[4], svodki[6][7])
    check_utt = simple_empty_soul(test[6],   svodki[7][7])   
      
    
    return info, svodki,self_defense, (check_sam,check_mtk,check_che,check_mtc,check_nvds,check_trans,check_nst,check_utt)
    


In [19]:
def super_test():
    
    """
    
    locomotion[0] - малая
    locomotion[1] - общая
    """
        
    vision = coub_02()
    info, svodki,self_defense,check = n9(vision)   
    
    #блок вывода профиля безопасности           
    
    self_defense[0].to_html('C:/Users/Battlestation/ural/характер_исполнения_учётной_политики.html')    
    self_defense[1].to_html('C:/Users/Battlestation/ural/характер_простоя_по_метрикам_безопасности.html')  
    self_defense[2].to_html('C:/Users/Battlestation/ural/колличестванная_оценка_.html') 
    
    # 08.04.2020 дополнения к фукнционалу
    
    self_defense_solo = self_defense[3].copy()
    
    self_defense_solo = self_defense[3].set_index('Поля рем.листов')
    
    solo_utt = self_defense_solo[self_defense_solo.columns[0:2]]
    solo_mtk = self_defense_solo[self_defense_solo.columns[2:4]]
    solo_mtc = self_defense_solo[self_defense_solo.columns[4:6]]
    solo_che = self_defense_solo[self_defense_solo.columns[6:8]]
    solo_tra = self_defense_solo[self_defense_solo.columns[8:10]]
    solo_nst = self_defense_solo[self_defense_solo.columns[10:12]]
    solo_nvds = self_defense_solo[self_defense_solo.columns[12:14]]
    solo_sam = self_defense_solo[self_defense_solo.columns[14:16]]  
    
    solo_utt.to_html('C:/Users/Battlestation/ural/характер_исполнения_учётной_политики_УТТ.html') 
    solo_mtk.to_html('C:/Users/Battlestation/ural/характер_исполнения_учётной_политики_МТК.html') 
    solo_mtc.to_html('C:/Users/Battlestation/ural/характер_исполнения_учётной_политики_МТС.html') 
    solo_che.to_html('C:/Users/Battlestation/ural/характер_исполнения_учётной_политики_ЧЕР.html') 
    solo_tra.to_html('C:/Users/Battlestation/ural/характер_исполнения_учётной_политики_ТРА.html') 
    solo_nst.to_html('C:/Users/Battlestation/ural/характер_исполнения_учётной_политики_НСТ.html') 
    solo_nvds.to_html('C:/Users/Battlestation/ural/характер_исполнения_учётной_политики_НВДС.html') 
    solo_sam.to_html('C:/Users/Battlestation/ural/характер_исполнения_учётной_политики_САМ.html')     
     
    
    check[0].to_html('C:/Users/Battlestation/ural/3х_летние_магины_которые_не_сломались_САМ.html') 
    check[1].to_html('C:/Users/Battlestation/ural/3х_летние_магины_которые_не_сломались_МТК.html') 
    check[2].to_html('C:/Users/Battlestation/ural/3х_летние_магины_которые_не_сломались_ЧЕР.html') 
    check[3].to_html('C:/Users/Battlestation/ural/3х_летние_магины_которые_не_сломались_МТС.html') 
    check[4].to_html('C:/Users/Battlestation/ural/3х_летние_магины_которые_не_сломались_НВДС.html') 
    check[5].to_html('C:/Users/Battlestation/ural/3х_летние_магины_которые_не_сломались_ТРАНС.html') 
    check[6].to_html('C:/Users/Battlestation/ural/3х_летние_магины_которые_не_сломались_НСТ.html') 
    check[7].to_html('C:/Users/Battlestation/ural/3х_летние_магины_которые_не_сломались_УТТ.html') 
    
    # что бы не перегружать генератор аниме надо запускать после коуб и ня
    
    anime = no_future()
    
    # пользуйтесь self_defense[2].loc['САМ'] работе с обзорной сводкой безопасности
    
    # дополнение 21_05_2020
    
    coub = ['Худшие по соблюдение учётной политики',
        'Худшие по простою новой техники',
       'Худшие по простою коррупционного характера',
       'Худшие по простою сомнительного характера',
       'Худшие по колличеству ТС в простое коррупционного характера',
       'Худшие по коллиеству ТС в простое сомнительного характера',
       'Худшие по колличеству ТС в простое новой техники',
       ]

    series = {"Компания":[self_defense[2].loc[self_defense[2]['НАРУШЕНИЕ УЧЁТА %'].idxmax()].name,
          self_defense[2].loc[self_defense[2]['ППРОСТОЙ НОВЫЕ ТС %'].idxmax()].name,
          self_defense[2].loc[self_defense[2]['ПРОСТОЙ КОРРУПЦИЯ %'].idxmax()].name,
          self_defense[2].loc[self_defense[2]['ПРОСТОЙ МАХИНАЦИИ %'].idxmax()].name,
          self_defense[2].loc[self_defense[2]['Число ТС по коррупции'].idxmax()].name,
          self_defense[2].loc[self_defense[2]['Число ТС по махинациям'].idxmax()].name,
          self_defense[2].loc[self_defense[2]['Число закупленных ТС '].idxmax()].name],
          
         
         "Числа" :[self_defense[2]['НАРУШЕНИЕ УЧЁТА %'].max(),
                  self_defense[2]['ППРОСТОЙ НОВЫЕ ТС %'].max(),
                  self_defense[2]['ПРОСТОЙ КОРРУПЦИЯ %'].max(),
                  self_defense[2]['ПРОСТОЙ МАХИНАЦИИ %'].max(),
                  self_defense[2]['Число ТС по коррупции'].max(),
                   self_defense[2]['Число ТС по махинациям'].max(),
                  self_defense[2]['Число закупленных ТС '].max()]
         }    

    tablet_bad = pd.DataFrame(index = coub,data=series) 

    coub = ['Лучшие по соблюдение учётной политики',
        'Лучшие по простою новой техники',
       'Лучшие по простою коррупционного характера',
       'Лучшие по простою сомнительного характера',
       'Лучшие по колличеству ТС в простое коррупционного характера',
       'Лучшие по коллиеству ТС в простое сомнительного характера',
       'Лучшие по колличеству ТС в простое новой техники',
       ]

    series = {"Компания":[self_defense[2].loc[self_defense[2]['НАРУШЕНИЕ УЧЁТА %'].idxmin()].name,
          self_defense[2].loc[self_defense[2]['ППРОСТОЙ НОВЫЕ ТС %'].idxmin()].name,
          self_defense[2].loc[self_defense[2]['ПРОСТОЙ КОРРУПЦИЯ %'].idxmin()].name,
          self_defense[2].loc[self_defense[2]['ПРОСТОЙ МАХИНАЦИИ %'].idxmin()].name,
          self_defense[2].loc[self_defense[2]['Число ТС по коррупции'].idxmin()].name,
          self_defense[2].loc[self_defense[2]['Число ТС по махинациям'].idxmin()].name,
          self_defense[2].loc[self_defense[2]['Число закупленных ТС '].idxmin()].name],
          
         
         "Числа" :[self_defense[2]['НАРУШЕНИЕ УЧЁТА %'].min(),
                  self_defense[2]['ППРОСТОЙ НОВЫЕ ТС %'].min(),
                  self_defense[2]['ПРОСТОЙ КОРРУПЦИЯ %'].min(),
                  self_defense[2]['ПРОСТОЙ МАХИНАЦИИ %'].min(),
                  self_defense[2]['Число ТС по коррупции'].min(),
                   self_defense[2]['Число ТС по махинациям'].min(),
                  self_defense[2]['Число закупленных ТС '].min()]
         }    

    tablet_good = pd.DataFrame(index = coub,data=series) 

    call_me_your_name = list(tablet_bad.Компания.value_counts().index.values.tolist()) 

    coub = ['кандидат на ревизию №1',
        'кандидат на ревизию №2',
        'кандидат на ревизию №2']

    series = {"Компания": [call_me_your_name[0],call_me_your_name[1],call_me_your_name[2]]}


    tablet_end = pd.DataFrame(index = coub,data=series) 
    
    # вывод по self_defense[4,5,6]
    
    tablet_bad.to_html('C:/Users/Battlestation/ural/Собственная_безопасность_ауйтсайдеры.html')   
    tablet_good.to_html('C:/Users/Battlestation/ural/Собственная_безопасность_лучшие.html') 
    tablet_end.to_html('C:/Users/Battlestation/ural/Рекомендательная_система.html')  
    
    def motion(df):
    
        series_1 = df[1][0][4].класс.value_counts()
        series_2 = df[2][0][4].класс.value_counts()
        series_3 = df[3][0][4].класс.value_counts()
        series_4 = df[4][0][4].класс.value_counts()
        series_5 = df[5][0][4].класс.value_counts()
        series_6 = df[6][0][4].класс.value_counts()
        series_7 = df[7][0][4].класс.value_counts()
        
        tablet_1 = pd.DataFrame(data=series_1)  
        tablet_2 = pd.DataFrame(data=series_2)  
        tablet_3 = pd.DataFrame(data=series_3)  
        tablet_4 = pd.DataFrame(data=series_4)  
        tablet_5 = pd.DataFrame(data=series_5)  
        tablet_6 = pd.DataFrame(data=series_6)  
        tablet_7 = pd.DataFrame(data=series_6)  
        
        
        tablet_1.rename(columns={'класс': 'НСТ'},inplace=True)
        tablet_2.rename(columns={'класс': 'ТРАНС'},inplace=True)
        tablet_3.rename(columns={'класс': 'НВДС'},inplace=True)
        tablet_4.rename(columns={'класс': 'ЧЕР'},inplace=True)
        tablet_5.rename(columns={'класс': 'МТС'},inplace=True)
        tablet_6.rename(columns={'класс': 'МТК'},inplace=True)
        tablet_7.rename(columns={'класс': 'САМ'},inplace=True)
        
        loco = pd.concat([tablet_1,tablet_2,tablet_3,tablet_4,tablet_5,tablet_6,tablet_7], axis=1)  
    
        loco['Метрики'] = loco.index
    
        loco['Метрики'] = loco['Метрики'].astype('str')
        loco['Метрики'] = loco['Метрики'].map(lambda x: "Класс ТС: "+ x )
    
        collist_1 = ['Метрики','НСТ', 'ТРАНС', 'НВДС', 'ЧЕР', 'МТС', 'МТК', 'САМ', ]
    
        loco= loco.reindex(columns=collist_1)
    
        collist_1 = ['Метрики','МТК', 'МТС', 'НВДС', 'НСТ', 'САМ', 'ТРАНС', 'УТТ', 'ЧЕР']
    
        locomotion = pd.concat([info[0],loco], axis=0)  
    
        locomotion = locomotion.reindex(columns=collist_1)
        
        return (loco,locomotion)
    
    locomotion = motion(anime)
    
    return info, svodki,self_defense,check,anime,locomotion,(tablet_bad, tablet_good,tablet_end)
    
    

In [20]:
# 01_06_2020

In [21]:
def simple_n(df_n,rout,n,warp):  
    
    # откликаеться на test = simple(df_n[0][0],rout[0],0)
     
    # [df_n] - датафрейм который мы передаём в фукнцию, передавать надо [0][0], [1][0], [2][0] и т.д
    # rout - мартшрут к записям о новых машинах 
    # [n] - номер варпа
    
    """
    Математика

    .mean() - ариф.сред
    .median() - медиана

    Число_операций == .count()
    Среднее_время_ремонта == .mean()
    Стандартное_отклонение == .std()
    Максимальное_время_ремонта == .max()

    Старта цикла == [цикл / число_машин] > 1 < 2
    Устойчивый цикл == [цикл/ число_машин] > 2

    Логика

    Постоянство_признак_мастерства ==  .mean() > .strd()

    Лень_матушка == .mean() + .std() > .max()

    Результат

    Оценка_процесса == .Постоянство_признак_мастерства(Проблема) ~ Лень_матушка(Проблема)
    Критические_проблемы == Оценка_процесса(процесс>2)

    """
    for pony in warp[n]: # расширить вывод. сделано

        # подаём уникальное имя машины с гаражным номером в генератор, получаем датафрейм в отношении одной машины

        name = pony

        df = df_n[df_n['Unic_gar']==name] 

        # читаем файл с закупками новых машин по обществу

        car_in = pd.read_excel(rout, index_col=0,
                        dtype={'Name': str, 'Value': float})

        # храним гаражный номер машины для сводной таблицы.

        number = df['_ГарНомер'].values        

        # сбрасываем unnamed в файле и переименовываем инв. номер в признак _gar

        car_in = car_in.drop(car_in.columns[car_in.columns.str.contains('unnamed', case = False)], axis = 1)
            
        # новые файлы иногда поступаю с дубликами записей - отпраялем дубликаты в варп
            
        car_in.rename(columns={car_in.columns[0]:'_gar'},inplace=True)
        car_in.rename(columns={car_in.columns[1]:'VIN'},inplace=True)
        car_in.rename(columns={car_in.columns[2]:'год_выпуска'},inplace=True)
        car_in.rename(columns={car_in.columns[4]:'Поставщик'},inplace=True)
        car_in.rename(columns={car_in.columns[5]:"Цена без НДС"}, inplace=True)     
        
        car_in = car_in.drop_duplicates(['_gar'], keep='last')
        
        # иногда в ремонтных листах ГарНомер записан через жопу , приводим к int64
        
        # отключено  в UTT гар.номера пишут с буквой на конце
        
        if n >0:
            df['_ГарНомер'] = df['_ГарНомер'].astype('float')
            df['_ГарНомер'] = df['_ГарНомер'].astype('int64')            
            
        # иногда год выпуска записан через жопу
            
        try:
            car_in = car_in.sort_values('год_выпуска',ascending=False)
        except KeyError:
            car_in = car_in.rename(columns={'Дата выпуска ': 'год_выпуска'})
            car_in['год_выпуска'] = car_in['год_выпуска'].astype('Datetime64')
        
        # проверяем датафрейм из генератора на принадлежность к списку новой техн
        
        eva_04 = df.loc[df['_ГарНомер'].isin(car_in._gar.values)]
                                             
        # в случае, если имеем дело с новой машиной возвращаем гаражный номер, vin, год выпуска, цены, поставщщ
        
        if len(eva_04) >=1:
            car_list = car_in.loc[car_in['_gar'].isin(eva_04._ГарНомер.values)]
            car_list_vin = car_list.VIN.values # 19.03.2020 перестал оборачивать в лист дома
            car_list_year = list(car_list['год_выпуска'].values)
            car_list_price = list(car_list['Цена без НДС'].values)
            car_list_registration = list(car_list['Дата принятия к учету'].values)
            car_list_seller = list(car_list.Поставщик.values)
        else:
            car_list = None
            car_list_vin = None
            car_list_year = None
            car_list_price = None
            car_list_registration = None
            car_list_seller = None
            
        # принт выведен в vision 

        print('Это новая машина, vin: ', car_list_vin," ", 
                      'год_выпуска: ', car_list_year,'\n','\n',

                      'цена: ', car_list_price,
                      "поставщик: ",  car_list_seller,'\n',
                      'Дата принятия к учёту: ', car_list_registration, sep='')

        # в случае, если датафрейм с машиной содержит более 2х записей - принимаем его в работу, это 2 поломки в год 
        # в противном случае выводим предупреждение, else в самом низу няша, если else значит ломается раз в год

        """size - считает каждую ремонтную операцию уникальной, как правило size больше чем кол-во рем.листов
            по причине того, что в одном ремонтном листе может быть записано несколько различных ремонтных операций"""

        size = df.Операции.nunique()      

        """true_size - считает каждый ремонтный лист уникальным эпизодом ремонта, как правило true_size меньше size"""

        true_size = len(df._Номер.unique())

        # float_size считает количество операций на один эпизод

        try:
            float_size = size/len(df)
        except ZeroDivisionError:
            float_size = 0
            
        float_size = float('{:.3f}'.format(float_size))

        """math - предназначен для поиска интервалов между эпизодами,
            math - считает каждый эпизод уникальным, вне зависимости от кол-ва ремонтных операций в эпизоде
            math - оперирует метрикой цикличности range - дистанцией в днях от первого рем.листа до второго и т.д"""

        # второй бэкап, чтобы не трогать исходную таблицу с машиной до преобразования

        me = df.copy()

        # нужно добавить в me фильтр только по уникальным операциям - иначе range будет лишен смысла в выводе.

        me = me.drop_duplicates(['_Номер'], keep='last')

        #  сортировка дат в календарном порядке, сначала первый лист в году потом последний   
        
        """ 
        18.03.2020 отключил дома
        
        me['_Дата'] = me['_Дата'].astype('datetime64')
        
        13.04 неожиданным образом это вызвало конфликт при работе с нст!
        
        возвращаею обратно, запускаю тест и это сразу вызвало исключение nontype obj is not interable
        
        раз в лоб не получаеться - убираю, пробую через трай
        
        """
        me = me.sort_values(by="_Дата", ascending = True)

        # формирование признака range - дистанция, в днях между ремонтными листами начиная с первого
        
        # ниже этого места едут остальные 10 тетрадок
        
        try:
            me['range'] = me._Дата.diff().dt.days
            me = me.set_index("_Дата")            
        except TypeError:
            me['_Дата'] = me['_Дата'].astype('datetime64')
            me['range'] = me._Дата.diff().dt.days
            me = me.set_index("_Дата") 
            
        #  без копирования  не едет me.range.fillna(me.range.mean())    
            
        test = me.copy()
                        
        test['math'] = me.range
        test['math'] = test['math'].fillna(test['math'].mean())
            
        # всё лишнее пока отправим на склад.
        
        test = test.drop(['_Орг', '_Номер', '_ТС', '_ГосНомер', '_ВидОбслуживания',
       '_ДатаНАч', '_ДатаКон', '_ГруппаРабот', '_КатегорияРабот', '_Работа',
       '_ФиоРММ', 'Операции', 'Машины', 'Unic_gar', 'result' ],axis=1)
        
        # необходимое учитывать что math иногда содержит нули ввиду того что эпизод ремонта был только 1.
        
        
        return test

In [22]:
def coub_04(x,n,w): 
    
    """исправленое 01.04.2020 - не ехелаа без передачи аргумент влоб
    
       c coub() внутри считаеться меделлно - но другого пути пока не вижу нет"""

    one = pd.DataFrame() # кеш
    
    df_n,warp,rout = coub() # 09_06_2020 возможно я лишний раз дёргая препроцессинг этим обращением
        
    def call(one,df_n,warp,rout):    # передача кеша, указатель на x - датафрейм, n - путь к файлу, 0 - номер варпа
        try:
            tablet = simple_n(df_n[x][0],rout[n],w,warp)             # запуска фукнции по кругу
            vision = pd.concat([one,tablet])                # коннетация
            one = vision.copy()                           # копирование  (кеширование не работает без копирования)
            return tablet,vision,one
        except ValueError:
            next
        except AttributeError:
            next
        except TypeError:
            print('в базе данных кончились машины - алгоритм отработал все объекты счисления') 

    def car_number(df):
        count = df.Unic_gar.nunique(dropna=True)  # подсчёт уникальных номеров и
        return count

    op_time = car_number(df_n[x][0]) # формирование длинны кеша
        
    for _ in range(op_time):
        tablet,vision,one = call(one,df_n,warp,rout)    # служебная
    
    return vision

In [23]:
def coub_05():   
    
    """13.03.2020 работает"""
    
    """16.03.2020 после преобразования  в 
    
       df['_ГарНомер'] = df['_ГарНомер'].astype('float')
       df['_ГарНомер'] = df['_ГарНомер'].astype('int64') 
            
       всё заработало кроме нвдс"""
    
    utt = coub_04(0,0,0)
    
    nst = coub_04(1,1,1)  
    
    trans = coub_04(2,2,2)    
    nvds = coub_04(3,3,3)
    cher = coub_04(4,4,4)
    mtc = coub_04(5,5,5)
    mtk = coub_04(6,6,6)    
    sam = coub_04(7,7,7) 
    
    # 25.03.2020 добавлено промежуточное хранилище под vision
    
    utt.to_csv('C:/Users/Battlestation/ural/vision_utt_me_n.csv', 
                  encoding='utf-8', index=False)
    nst.to_csv('C:/Users/Battlestation/ural/vision_nst_me_n.csv', 
                  encoding='utf-8', index=False)
    trans.to_csv('C:/Users/Battlestation/ural/vision_trans_me_n.csv', 
                  encoding='utf-8', index=False)
    nvds.to_csv('C:/Users/Battlestation/ural/vision_nvds_me_n.csv', 
                  encoding='utf-8', index=False)
    cher.to_csv('C:/Users/Battlestation/ural/vision_cher_me_n.csv', 
                  encoding='utf-8', index=False)
    mtc.to_csv('C:/Users/Battlestation/ural/vision_mtc_me_n.csv', 
                  encoding='utf-8', index=False)
    mtk.to_csv('C:/Users/Battlestation/ural/vision_mtk_me_n.csv', 
                  encoding='utf-8', index=False)
    sam.to_csv('C:/Users/Battlestation/ural/vision_sam_me_n.csv', 
                  encoding='utf-8', index=False)
    
    return (utt,nst,trans,nvds,cher,mtc,mtk,sam)




In [24]:
def work_night():
    
    # # df1['g']=df1.groupby('A').cumcount() возможно пригодиться
    
    info, svodki,self_defense,check,anime,locomotion,tablet = super_test()
    
    vision = coub_05()
    
    # (utt,nst,trans,nvds,cher,mtc,mtk,sam)
    
    utt = vision[0].copy()
    utt_log = utt.dropna(subset=['math'])    
    res_log_utt = utt_log.join(anime[0][0][4].set_index('_ГарНомер')[['класс','активность','надёжность','простой']], on='_ГарНомер')
    
    nst = vision[1].copy()
    nst_log = nst.dropna(subset=['math'])    
    res_log_nst = nst_log.join(anime[1][0][4].set_index('_ГарНомер')[['класс','активность','надёжность','простой']], on='_ГарНомер')
    
    trans = vision[2].copy()
    trans_log = trans.dropna(subset=['math'])    
    res_log_trans = trans_log.join(anime[2][0][4].set_index('_ГарНомер')[['класс','активность','надёжность','простой']], on='_ГарНомер')
    
    # 27_08_2020 нашёл ошибку которая портила nvds 
    # было nvds_log = nst.dropna(subset=['math']) 
    
    nvds = vision[3].copy()
    nvds_log = nvds.dropna(subset=['math'])    
    res_log_nvds = nvds_log.join(anime[3][0][4].set_index('_ГарНомер')[['класс','активность','надёжность','простой']], on='_ГарНомер')
    
    cher = vision[4].copy()
    cher_log = cher.dropna(subset=['math'])    
    res_log_cher = cher_log.join(anime[4][0][4].set_index('_ГарНомер')[['класс','активность','надёжность','простой']], on='_ГарНомер')
    
    mtc = vision[5].copy()
    mtc_log = mtc.dropna(subset=['math'])    
    res_log_mtc = mtc_log.join(anime[5][0][4].set_index('_ГарНомер')[['класс','активность','надёжность','простой']], on='_ГарНомер')
    
    mtk = vision[6].copy()
    mtk_log = mtk.dropna(subset=['math'])    
    res_log_mtk = mtk_log.join(anime[6][0][4].set_index('_ГарНомер')[['класс','активность','надёжность','простой']], on='_ГарНомер')
    
    sam = vision[7].copy()
    sam_log = sam.dropna(subset=['math'])    
    res_log_sam = sam_log.join(anime[7][0][4].set_index('_ГарНомер')[['класс','активность','надёжность','простой']], on='_ГарНомер')
    
    res_log_utt.to_csv('C:/Users/Battlestation/ural/utt_math.csv', 
                  encoding='utf-8', index=True)
    res_log_nst.to_csv('C:/Users/Battlestation/ural/nst_math.csv', 
                  encoding='utf-8', index=True)
    res_log_trans.to_csv('C:/Users/Battlestation/ural/trans_math.csv', 
                  encoding='utf-8', index=True)
    res_log_nvds.to_csv('C:/Users/Battlestation/ural/nvds_math.csv', 
                  encoding='utf-8', index=True)
    res_log_cher.to_csv('C:/Users/Battlestation/ural/cher_math.csv', 
                  encoding='utf-8', index=True)
    res_log_mtc.to_csv('C:/Users/Battlestation/ural/mtc_math.csv', 
                  encoding='utf-8', index=True)
    res_log_mtk.to_csv('C:/Users/Battlestation/ural/mtk_meath.csv', 
                  encoding='utf-8', index=True)
    res_log_sam.to_csv('C:/Users/Battlestation/ural/sam_math.csv', 
                  encoding='utf-8', index=True)
    
    # 17_07_2020нашёл ошибки в return  - не хватало res_log_trans
    
    return info, svodki,self_defense,check,anime,locomotion,tablet,(res_log_utt,res_log_nst,res_log_trans,res_log_nvds,res_log_cher,res_log_mtc,res_log_mtk,res_log_sam)

In [24]:
# ural_manga_coub_14_06_2020 завершённая версия, единственный косяк - возможно лишняя нагрузка. 

# откликаеться на info, svodki,self_defense,check,anime,locomotion,tablet,no_life = work_night()

In [ ]:
info, svodki,self_defense,check,anime,locomotion,tablet,no_life = work_night()